# Introduction
This topic deals with *k-dimensional trees* , or *kd trees*. This data structure allows one to efficiently
find, among a set of points of R<sup>k</sup>, the nearest neighbor of a given point. 

Kd trees are similar to binary search trees, except that the order used to distribute the descendants
of a node *n* between the left and right subtrees depends on the depth of *n* in the tree: the nodes
are sorted alternately according to each of their *k* coordinates.

More formally, a *k*-dimensional tree is a binary tree whose nodes each contain a point of R<sup>k</sup>.
Moreover, if p = (*p0, p1, . . . , pk−1*) is a point placed in a node *n* at depth *i* and denoting *r* the
remainder of the Euclidean division of *i* by *k*:

- for any point q = (*q0, q1, . . . , qk−1*) placed in the left subtree from *n*, we have q<sub>r</sub> < p<sub>r</sub>,
- for any point q = (*q0, q1, . . . , qk−1*) of the right subtree, we have q<sub>r</sub> ≥ p<sub>r</sub>.

(We agree that the depth of the root is 0.)

Conventionally, a tree or subtree is represented by its root node. A node contains a point of R<sup>k</sup>, of
type `double[]`, and pointers to the left and right subtrees. Each node also stores its depth in the
tree (field depth).

```java
class KDTree {
    int depth; double[] point; KDTree left; KDTree right;
    // construct a leaf
    KDTree(double[] point, int depth) { 
        this.point = point; 
        this.depth = depth;
    }
}
```

As in the course, the empty tree is represented by `null`.


# 1 Insertion

We want to add a method that inserts a point a into a tree while preserving the property defining kd trees. To do this, we need to know whether to place the point a in the left subtree or the right subtree.

## 1.1 Comparison

In the class KDTree, complete the method `boolean compare(double[] a))` that returns `true` whether the point a should be inserted into the right subtree, and `false` whether it should be inserted into the left subtree.

In order to reuse it later, we can isolate in a separate method the calculation of the difference in coordinates which is involved in the choice.

Test your code with `TestCompare`.

## 1.2 Insertion
In the class KDTree, complete the method KDTree `insert(KDTree tree, double[] a)` that adds a node containing the point a to the tree **tree** and returns the root of the resulting tree. The insertion method must preserve the property defining kd trees. (Be careful to correctly set the depth of the inserted point!) We are allowed to insert multiple copies of the same point.

Test your code with `TestInsert` and `InteractiveClosest`. The window produced by `InteractiveClosest` represents the 2-dimensional tree obtained after successively inserting 50 points chosen uniformly in [0, 1] × [0, 1]. Type `'+'` or `'-'` to have more or fewer points.

Here is for example what we get with 1016 points:

<div>
<img src="1016.png" width="500"/>
</div>

# 2 Nearest point

We are now interested in finding the point of a kd tree which is closest to a given point a, in the sense of Euclidean distance.

## 2.1 Distance

Complete the method `double sqDist(double[] a, double[] b)` that calculates the square of the distance between two points.

In the class `TestSqDist`, write some tests to verify that **sqDistit** works correctly.

For example, the square of the distance between the points (−1, 1) and (1, −1) is equal to 8. Test some other representative examples of the possible inputs. Remember that sqDist must work in all dimensions ( k = 0, k = 1, k = 1000, . . . ).

## 2.2 Naive version

The naive algorithm for finding the nearest point simply consists of traversing all the points in the tree, maintaining in a variable **champion** the closest point encountered so far.

Complete the method `static double[] closestNaive(KDTree tree, double[] a)` that returns the closest point to the point **a** in the tree **tree**, or `null` if the tree is empty. 

(An auxiliary method may be useful.) 

Test your code with `TestClosestNaive`.

## 2.3 A more efficient algorithm

The complexity of the previous method `closestNaive` is linear in the size of the tree. Much better can be done using the kd tree structure.

We observe that for any subtree t depth-dependent i in our kd tree, the points in the left subtree of t are separated from the points in the right subtree of t by the cutoff hyperplane defined by 

**x[i%k] = t.point[i%k].**

To find the nearest neighbor of a point a in t, we can therefore start by looking for the nearest neighbor p in the subtree of t located on the same side of the cut-off hyperplane as a. If a is closer to than p to the cut-off hyperplane, then there is no point in exploring the other subtree of t(situation 1). Otherwise, we must also consider t.point and the points in the right subtree of t (situation 2).

Complete the methods:

- `static double[] closest(KDTree tree, double[] a, double[] champion)` which searches for the closest point to point a in the set consisting of the tree tree and champion as described above;
  
- `static double[] closest(KDTree tree, double[] a)` which returns the closest point to a; if the tree is empty, we return null.

At the start of each recursive call, the three-parameter version of closest should call `InteractiveClosest.trace(tree.point, champion)` with **champion** the closest point known at that point as the parameter (see the provided code).

TestSqDist.testSqDist your code with the classes `TestClosestOptimized` and `InteractiveClosest`.

The class `InteractiveClosest` allows you to visualize the algorithm’s progress. Click to find the point closest to the cursor. Hold down to continuously refresh. The nodes traversed are displayed in blue, and the successive champions in orange. If the orange line is very broken (say, more than 10 points), there is a problem.

<div>
<img src="algvis.png" width="500"/>
</div>

# 3 Selecting an optimized color palette
For common applications, the colors of images viewed on a screen are encoded with 24 bits, 8 bits for each of the three colors red, green, and blue, which gives one point. To compress an image, the GIF image format selects 256 colors, from the 16777216 possible, to best represent the image. The method of fixing 256 colors independently of the image, such as the `web palette`, gives very poor results. It is necessary to select a color palette appropriate to the image.

The goal of this part is to write the function palette that returns an object of type `Vector<double[]>` containing a prescribed number of points capable of approximating the points of a given kd tree.

**Figure 1:** witness photo
<div>
<img src="Figure 1.png" title="Figure 1" width="500"/>
</div>

**Figure 2:** web palette
<div>
<img src="Figure 2.png" title="Figure 2" width="500"/>
</div>

**Figure 3:** optimized palette
<div>
<img src="Figure 3.png" title="Figure 3" width="500"/>
</div>

## 3.1 Some auxiliary functions

Complete the following methods

- `int size(KDTree tree)` which returns the number of points in the tree;
- `void sum(KDTree tree, double[] acc)` which calculates the sum of the points of the tree, adds it to **acc**;
- `double[] average(KDTree tree)` which returns the isobarycentre point (i.e. the average value) of the points in the tree, or `null` for an empty tree.

Test your code with `TestAverage`.

## 3.2 The method palette
The palette is composed from a 3D tree containing the colors of 20, 000 pixels randomly chosen from the image. The 256 colors will be obtained by calculating the isobarycentre of the points of well-chosen subtrees.

Complete the method `static Vector<double[]> palette(KDTree tree, int maxpoints)` that returns an array of **maxpoints** colors based on tree. Several strategies are possible.

Test your code with the class `ColorPalette`. A computation time of more than 1 second or a score above 10 indicates a problem. Your teachers have gotten down to around 4.3. Share your method if you do less!

It’s okay to experiment. Start with a naive method for choosing `maxpoints` subtrees whose iso- barycenters we’ll take, then refine. At first, consider maxpoints in an order of magnitude rather than a strict constraint. Note that in our application, the three-dimensional tree **tree** from which we extract a palette is not necessarily balanced (it’s a random tree).

# Perspectives

Beyond palette selection, dither methods are essential to avoid the flatness inherent in methods that replace a color with its nearest neighbor in a fixed palette. But that’s another story...

# Code

## Dependencies

In [1]:
import java.awt.Color;
import java.awt.BasicStroke;
import java.awt.Component;
import java.awt.FileDialog;
import java.awt.Font;
import java.awt.FontMetrics;
import java.awt.Graphics;
import java.awt.Graphics2D;
import java.awt.geom.Arc2D;
import java.awt.geom.Ellipse2D;
import java.awt.geom.GeneralPath;
import java.awt.geom.Line2D;
import java.awt.geom.Rectangle2D;
import java.awt.FileDialog;
import java.awt.Toolkit;
import java.awt.event.ActionEvent;
import java.awt.event.ActionListener;
import java.awt.event.KeyEvent;
import java.awt.Image;
import java.awt.image.DirectColorModel;
import java.awt.image.WritableRaster;
import java.awt.image.BufferedImage;
import java.awt.MediaTracker;
import java.awt.RenderingHints;
import java.awt.Toolkit;
import java.awt.event.ActionEvent;
import java.awt.event.ActionListener;
import java.awt.event.MouseEvent;
import java.awt.event.MouseListener;
import java.awt.event.MouseMotionListener;
import java.awt.event.KeyEvent;
import java.awt.event.KeyListener;

import java.util.Vector;
import java.util.Arrays;
import java.util.LinkedList;
import java.util.ArrayList;
import java.util.TreeSet;
import java.util.Random;
import java.util.Vector;
import java.util.Locale;
import java.util.Collections;

import java.io.File;
import java.io.IOException;
import java.net.URL;
import java.net.MalformedURLException;
import javax.imageio.ImageIO;

import javax.swing.ImageIcon;
import javax.swing.JFrame;
import javax.swing.JLabel;
import javax.swing.JMenu;
import javax.swing.JMenuBar;
import javax.swing.JMenuItem;
import javax.swing.JPanel;
import javax.swing.KeyStroke;
import javax.swing.SwingUtilities;

## Picture

In [2]:
// Picture.java
public final class Picture implements ActionListener {
	private BufferedImage image; // the rasterized image
	private JFrame frame; // on-screen view
	private String filename; // name of file
	private boolean isOriginUpperLeft = true; // location of origin
	private final int width, height; // width and height

	/**
	 * Creates a {@code width}-by-{@code height} picture, with {@code width} columns
	 * and {@code height} rows, where each pixel is black.
	 *
	 * @param width  the width of the picture
	 * @param height the height of the picture
	 * @throws IllegalArgumentException if {@code width} is negative or zero
	 * @throws IllegalArgumentException if {@code height} is negative or zero
	 */
	public Picture(int width, int height) {
		if (width <= 0)
			throw new IllegalArgumentException("width must be positive");
		if (height <= 0)
			throw new IllegalArgumentException("height must be positive");
		this.width = width;
		this.height = height;
		image = new BufferedImage(width, height, BufferedImage.TYPE_INT_RGB);
		// set to TYPE_INT_ARGB here and in next constructor to support transparency
	}

	/**
	 * Creates a new picture that is a deep copy of the argument picture.
	 *
	 * @param picture the picture to copy
	 * @throws IllegalArgumentException if {@code picture} is {@code null}
	 */
	public Picture(Picture picture) {
		if (picture == null)
			throw new IllegalArgumentException("constructor argument is null");

		width = picture.width();
		height = picture.height();
		image = new BufferedImage(width, height, BufferedImage.TYPE_INT_RGB);
		filename = picture.filename;
		isOriginUpperLeft = picture.isOriginUpperLeft;
		for (int col = 0; col < width(); col++)
			for (int row = 0; row < height(); row++)
				image.setRGB(col, row, picture.image.getRGB(col, row));
	}

	/**
	 * Creates a picture by reading an image from a file or URL.
	 *
	 * @param name the name of the file (.png, .gif, or .jpg) or URL.
	 * @throws IllegalArgumentException if cannot read image
	 * @throws IllegalArgumentException if {@code name} is {@code null}
	 */
	public Picture(String name) {
		if (name == null)
			throw new IllegalArgumentException("constructor argument is null");

		this.filename = name;
		try {
			// try to read from file in working directory
			File file = new File(name);
			if (file.isFile()) {
				image = ImageIO.read(file);
			}

			else {

				// resource relative to .class file
				URL url = getClass().getResource(filename);

				// resource relative to classloader root
				if (url == null) {
					url = getClass().getClassLoader().getResource(name);
				}

				// or URL from web
				if (url == null) {
					url = new URL(name);
				}

				image = ImageIO.read(url);
			}

			if (image == null) {
				throw new IllegalArgumentException("could not read image: " + name);
			}

			width = image.getWidth(null);
			height = image.getHeight(null);
		} catch (IOException ioe) {
			throw new IllegalArgumentException("could not open image: " + name, ioe);
		}
	}

	/**
	 * Creates a picture by reading the image from a PNG, GIF, or JPEG file.
	 *
	 * @param file the file
	 * @throws IllegalArgumentException if cannot read image
	 * @throws IllegalArgumentException if {@code file} is {@code null}
	 */
	public Picture(File file) {
		if (file == null)
			throw new IllegalArgumentException("constructor argument is null");

		try {
			image = ImageIO.read(file);
		} catch (IOException ioe) {
			throw new IllegalArgumentException("could not open file: " + file, ioe);
		}
		if (image == null) {
			throw new IllegalArgumentException("could not read file: " + file);
		}
		width = image.getWidth(null);
		height = image.getHeight(null);
		filename = file.getName();
	}

	/**
	 * Returns a {@link JLabel} containing this picture, for embedding in a
	 * {@link JPanel}, {@link JFrame} or other GUI widget.
	 *
	 * @return the {@code JLabel}
	 */
	public JLabel getJLabel() {
		if (image == null)
			return null; // no image available
		ImageIcon icon = new ImageIcon(image);
		return new JLabel(icon);
	}

	/**
	 * Sets the origin to be the upper left pixel. This is the default.
	 */
	public void setOriginUpperLeft() {
		isOriginUpperLeft = true;
	}

	/**
	 * Sets the origin to be the lower left pixel.
	 */
	public void setOriginLowerLeft() {
		isOriginUpperLeft = false;
	}

	/**
	 * Displays the picture in a window on the screen.
	 */
	public void show() {
		
		// create the GUI for viewing the image if needed
		if (frame == null) {
			frame = new JFrame();

			JMenuBar menuBar = new JMenuBar();
			JMenu menu = new JMenu("File");
			menuBar.add(menu);
			JMenuItem menuItem1 = new JMenuItem(" Save...   ");
			menuItem1.addActionListener(this);
			// use getMenuShortcutKeyMaskEx() in Java 10 (getMenuShortcutKeyMask()
			// deprecated)
			menuItem1.setAccelerator(
					KeyStroke.getKeyStroke(KeyEvent.VK_S, Toolkit.getDefaultToolkit().getMenuShortcutKeyMaskEx()));
			menu.add(menuItem1);
			frame.setJMenuBar(menuBar);

			frame.setContentPane(getJLabel());
			// frame.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);
			frame.setDefaultCloseOperation(JFrame.DISPOSE_ON_CLOSE);
			if (filename == null)
				frame.setTitle(width + "-by-" + height);
			else
				frame.setTitle(filename);
			frame.setResizable(false);
			frame.pack();
			frame.setVisible(true);
		}

		// draw
		frame.repaint();
	}

	/**
	 * Returns the height of the picture.
	 *
	 * @return the height of the picture (in pixels)
	 */
	public int height() {
		return height;
	}

	/**
	 * Returns the width of the picture.
	 *
	 * @return the width of the picture (in pixels)
	 */
	public int width() {
		return width;
	}

	private void validateRowIndex(int row) {
		if (row < 0 || row >= height())
			throw new IllegalArgumentException("row index must be between 0 and " + (height() - 1) + ": " + row);
	}

	private void validateColumnIndex(int col) {
		if (col < 0 || col >= width())
			throw new IllegalArgumentException("column index must be between 0 and " + (width() - 1) + ": " + col);
	}

	/**
	 * Returns the color of pixel ({@code col}, {@code row}) as a
	 * {@link java.awt.Color}.
	 *
	 * @param col the column index
	 * @param row the row index
	 * @return the color of pixel ({@code col}, {@code row})
	 * @throws IllegalArgumentException unless both {@code 0 <= col < width} and
	 *                                  {@code 0 <= row < height}
	 */
	public Color get(int col, int row) {
		validateColumnIndex(col);
		validateRowIndex(row);
		int rgb = getRGB(col, row);
		return new Color(rgb);
	}

	/**
	 * Returns the color of pixel ({@code col}, {@code row}) as an {@code int}.
	 * Using this method can be more efficient than {@link #get(int, int)} because
	 * it does not create a {@code Color} object.
	 *
	 * @param col the column index
	 * @param row the row index
	 * @return the integer representation of the color of pixel ({@code col},
	 *         {@code row})
	 * @throws IllegalArgumentException unless both {@code 0 <= col < width} and
	 *                                  {@code 0 <= row < height}
	 */
	public int getRGB(int col, int row) {
		validateColumnIndex(col);
		validateRowIndex(row);
		if (isOriginUpperLeft)
			return image.getRGB(col, row);
		else
			return image.getRGB(col, height - row - 1);
	}

	/**
	 * Sets the color of pixel ({@code col}, {@code row}) to given color.
	 *
	 * @param col   the column index
	 * @param row   the row index
	 * @param color the color
	 * @throws IllegalArgumentException unless both {@code 0 <= col < width} and
	 *                                  {@code 0 <= row < height}
	 * @throws IllegalArgumentException if {@code color} is {@code null}
	 */
	public void set(int col, int row, Color color) {
		validateColumnIndex(col);
		validateRowIndex(row);
		if (color == null)
			throw new IllegalArgumentException("color argument is null");
		int rgb = color.getRGB();
		setRGB(col, row, rgb);
	}

	/**
	 * Sets the color of pixel ({@code col}, {@code row}) to given color.
	 *
	 * @param col the column index
	 * @param row the row index
	 * @param rgb the integer representation of the color
	 * @throws IllegalArgumentException unless both {@code 0 <= col < width} and
	 *                                  {@code 0 <= row < height}
	 */
	public void setRGB(int col, int row, int rgb) {
		validateColumnIndex(col);
		validateRowIndex(row);
		if (isOriginUpperLeft)
			image.setRGB(col, row, rgb);
		else
			image.setRGB(col, height - row - 1, rgb);
	}

	/**
	 * Returns true if this picture is equal to the argument picture.
	 *
	 * @param other the other picture
	 * @return {@code true} if this picture is the same dimension as {@code other}
	 *         and if all pixels have the same color; {@code false} otherwise
	 */
	public boolean equals(Object other) {
		if (other == this)
			return true;
		if (other == null)
			return false;
		if (other.getClass() != this.getClass())
			return false;
		Picture that = (Picture) other;
		if (this.width() != that.width())
			return false;
		if (this.height() != that.height())
			return false;
		for (int col = 0; col < width(); col++)
			for (int row = 0; row < height(); row++)
				if (this.getRGB(col, row) != that.getRGB(col, row))
					return false;
		return true;
	}

	/**
	 * Returns a string representation of this picture. The result is a
	 * <code>width</code>-by-<code>height</code> matrix of pixels, where the color
	 * of a pixel is represented using 6 hex digits to encode the red, green, and
	 * blue components.
	 *
	 * @return a string representation of this picture
	 */
	public String toString() {
		StringBuilder sb = new StringBuilder();
		sb.append(width + "-by-" + height + " picture (RGB values given in hex)\n");
		for (int row = 0; row < height; row++) {
			for (int col = 0; col < width; col++) {
				int rgb = 0;
				if (isOriginUpperLeft)
					rgb = image.getRGB(col, row);
				else
					rgb = image.getRGB(col, height - row - 1);
				sb.append(String.format("#%06X ", rgb & 0xFFFFFF));
			}
			sb.append("\n");
		}
		return sb.toString().trim();
	}

	/**
	 * This operation is not supported because pictures are mutable.
	 *
	 * @return does not return a value
	 * @throws UnsupportedOperationException if called
	 */
	public int hashCode() {
		throw new UnsupportedOperationException("hashCode() is not supported because pictures are mutable");
	}

	/**
	 * Saves the picture to a file in either PNG or JPEG format. The filetype
	 * extension must be either .png or .jpg.
	 *
	 * @param name the name of the file
	 * @throws IllegalArgumentException if {@code name} is {@code null}
	 */
	public void save(String name) {
		if (name == null)
			throw new IllegalArgumentException("argument to save() is null");
		save(new File(name));
		filename = name;
	}

	/**
	 * Saves the picture to a file in a PNG or JPEG image format.
	 *
	 * @param file the file
	 * @throws IllegalArgumentException if {@code file} is {@code null}
	 */
	public void save(File file) {
		if (file == null)
			throw new IllegalArgumentException("argument to save() is null");
		filename = file.getName();
		if (frame != null)
			frame.setTitle(filename);
		String suffix = filename.substring(filename.lastIndexOf('.') + 1);
		if ("jpg".equalsIgnoreCase(suffix) || "png".equalsIgnoreCase(suffix)) {
			try {
				ImageIO.write(image, suffix, file);
			} catch (IOException e) {
				e.printStackTrace();
			}
		} else {
			System.out.println("Error: filename must end in .jpg or .png");
		}
	}

	/**
	 * Opens a save dialog box when the user selects "Save As" from the menu.
	 */
	@Override
	public void actionPerformed(ActionEvent e) {
		FileDialog chooser = new FileDialog(frame, "Use a .png or .jpg extension", FileDialog.SAVE);
		chooser.setVisible(true);
		if (chooser.getFile() != null) {
			save(chooser.getDirectory() + File.separator + chooser.getFile());
		}
	}
}

In [3]:
/**
 * Unit tests this {@code Picture} data type. Reads a picture specified by the
 * command-line argument, and shows it in a window on the screen.
 *
 * @param args the command-line arguments
 */	
Picture picture = new Picture("photo.jpg");
System.out.printf("%d-by-%d\n", picture.width(), picture.height());
picture.show();

992-by-744


## KDTree (code to complete)

In [4]:
// KDTree.java
public class KDTree {
	int depth;
	double[] point;
	KDTree left;
	KDTree right;

	KDTree(double[] point, int depth) {
		this.point = point;
		this.depth = depth;
	}

	boolean compare(double[] a) {
		int r = this.depth % a.length; // r = i%k
		if (r > point.length)
			return true;
		return a[r] >= point[r];
	}

	static KDTree insert(KDTree tree, double[] p) {
		if (tree == null)
			return new KDTree(p, 0);

		KDTree newTree = new KDTree(tree.point, tree.depth);
		newTree.right = tree.right;
		newTree.left = tree.left;

		KDTree current = newTree;
		while(true){
			if(current.compare(p)){
				if(current.right == null){
					current.right = new KDTree(p, current.depth + 1);
					break;
				}
				else
					current = current.right;
			}
			else{
				if(current.left == null){
					current.left = new KDTree(p, current.depth + 1);
					break;
				}
				else
					current = current.left;
			}
		}

		return newTree;
	}

	static double sqDist(double[] a, double[] b) {
		double result = 0.0;
		for (int i=0; i < a.length; i++){
			result += (a[i] - b[i])*(a[i] - b[i]);	
		}
		return result;
	}

	static double[] closestNaive(KDTree tree, double[] a, double[] champion) {
		throw(new Error("TODO"));
	}


	static double[] closestNaive(KDTree tree, double[] a) {
		if (tree == null)
			return null;

		double[] closest = tree.point;
		LinkedList<KDTree> queue = new LinkedList<KDTree>();
		queue.addLast(tree.left);
		queue.addLast(tree.right);
		while(!queue.isEmpty()){
			KDTree current = queue.pop();
			if (current == null)
				continue;
			queue.addLast(current.left);
			queue.addLast(current.right);
			closest = (sqDist(current.point, a) < sqDist(closest, a)) ? current.point : closest;
		}
		return closest;
	}

	static double[] closest(KDTree tree, double[] a, double[] champion) {
		if (tree == null)
			return champion;
		
		Stack<KDTree> stack = new Stack<KDTree>();
		stack.push(tree);
		while(!stack.isEmpty()){
			// hyperplane is current.point
			KDTree current = stack.pop();
			if (current == null) continue;
			InteractiveClosest.trace(current.point, champion);
			
			// false = left, true = right
			boolean left_right = current.compare(a); 
			KDTree next = left_right ? current.right : current.left;
			if (next == null) continue;				
			if (sqDist(a, next.point) < sqDist(a, champion))
				champion = next.point;

			// No consider other side
			if (sqDist(a, champion) < sqDist(a, current.point)){
				stack.push(next);
				continue;
			}

			// consider other side
			KDTree next2 = !(left_right) ? current.right : current.left;
			if (next2 == null) continue;
			if (sqDist(a, next2.point) < sqDist(a, champion))
				champion = next2.point;
			stack.push(next2);
			stack.push(next);	
		}
		return champion;
	}

	static double[] closest(KDTree tree, double[] a) {
		if (tree == null)
			return null;
		return closest(tree, a, tree.point);
	}

	static int size(KDTree tree) {
		if (tree == null)
			return 0;
		int result = 1;
		LinkedList<KDTree> queue = new LinkedList<KDTree>();
		queue.addLast(tree.left);
		queue.addLast(tree.right);
		while(!queue.isEmpty()){
			KDTree current = queue.pop();
			if (current == null)
				continue;
			queue.addLast(current.left);
			queue.addLast(current.right);
			result += 1;
		}
		return result;
	}

	static void sum(KDTree tree, double[] acc) {
		if (tree == null)
			return;
		for(int i=0; i < acc.length; i++){
			acc[i] += tree.point[i];	
		}
		LinkedList<KDTree> queue = new LinkedList<KDTree>();
		queue.addLast(tree.left);
		queue.addLast(tree.right);
		while(!queue.isEmpty()){
			KDTree current = queue.pop();
			if (current == null)
				continue;
			queue.addLast(current.left);
			queue.addLast(current.right);
			pointAdd(acc, current.point);
		}
	}

	static double[] average(KDTree tree) {
		if (tree == null)
			return null;
		double[] acc = new double[tree.point.length];
		sum(tree, acc);
		int size = size(tree);
		for (int i=0; i < acc.length; i++){
			acc[i] = acc[i] / size;
		}
		return acc;
	}

	/*static Vector<double[]> palette(KDTree tree, int maxpoints) {
		if (tree == null)
			return null;

		Vector<double[]> allPoints = getAllNonDupePoints(tree);
		Vector<double[]> result = new Vector<double[]>();
		allPoints.sort(new Comparator<double[]>() {
			public int compare(double[] a, double[] b) {
				return Double.compare(sqDist(a, new double[a.length]), sqDist(b, new double[b.length]));
			}
		});
		for(int i=0; i < allPoints.size() && result.size() < maxpoints; i += allPoints.size()/maxpoints ){
			result.add(allPoints.get(i));
		}
		return result;	
	}*/

	static Vector<double[]> palette(KDTree tree, int maxpoints) {
		if (tree == null)
			return null;

		int K = maxpoints;
		Vector<double[]> allPoints = getAllNonDupePoints(tree);
		Vector<double[]> allPointsCopy = new Vector<double[]>(allPoints);
		Collections.shuffle(allPointsCopy);
		Vector<Cluster> clusters = new Vector<Cluster>();
		for (int i = 0; i < K; i++) {
			Vector<double[]> clusterPoints = new Vector<double[]>();
			double[] centroid = allPointsCopy.get(i);
			clusterPoints.add(centroid);
			clusters.add(new Cluster(clusterPoints));
		}

		//k-means clustering
		boolean converged = false;
		while (!converged) {
			converged = true;
			for (double[] point : allPoints) {
				double minDist = Double.MAX_VALUE;
				int closestClusterIndex = -1;
				for (int j = 0; j < clusters.size(); j++) {
					Cluster cluster = clusters.get(j);
					double dist = sqDist(point, cluster.getCentroid());
					if (dist < minDist) {
						minDist = dist;
						closestClusterIndex = j;
					}
				}
				clusters.get(closestClusterIndex).addPoint(point);
			}

			for (Cluster cluster : clusters) {
				double[] oldCentroid = cluster.getCentroid();
				cluster.updateCentroid();
				double[] newCentroid = cluster.getCentroid();
				if (!Arrays.equals(oldCentroid, newCentroid)) {
					converged = false;
				}
				cluster.clearPoints(); // Clear points for next iteration
			}
		}
		Vector<double[]> result = new Vector<double[]>();
		for (int i = 0; i < clusters.size(); i++) {
			Cluster cluster = clusters.get(i);
			result.add(cluster.getCentroid());
		}
		return result;
		
	}

	private static void pointAdd(double[] a, double[] b) {
		for (int i=0; i < a.length; i++){
			a[i] += b[i];	
		}
	}

	private static Vector<double[]> getAllNonDupePoints(KDTree tree) {
		if (tree == null)
			return null;
			
		Vector<double[]> result = new Vector<double[]>();
		Set<KDTree> nonDupeTrees = new HashSet<>();
		LinkedList<KDTree> queue = new LinkedList<KDTree>();
		queue.addLast(tree.left);
		queue.addLast(tree.right);
		while(!queue.isEmpty()){
			KDTree current = queue.pop();
			if (current == null)
				continue;
			queue.addLast(current.left);
			queue.addLast(current.right);
			nonDupeTrees.add(current);
		}	
		for (KDTree t : nonDupeTrees) {
			result.add(t.point);
		}
		return result;
	}

	public String pointToString() {
		StringBuffer sb = new StringBuffer();
		sb.append("[");
		if (this.point.length > 0)
			sb.append(this.point[0]);
		for (int i = 1; i < this.point.length; i++)
			sb.append("," + this.point[i]);
		sb.append("]");
		return sb.toString();
	}

	@Override
	public boolean equals(Object o) {
		if (o == null)
			return false;
		if (o.getClass() != this.getClass())
			return false;
		KDTree that = (KDTree) o;
		if (this.point.length != that.point.length)
			return false;
		// similar
		if (sqDist(this.point, that.point) == 0.0)
			return true;
		return false;
	}
}

In [5]:
public class Cluster{
    private double[] centroid;
    private Vector<double[]> points;
    private int dimension;

    Cluster(Vector<double[]> points){
        this.dimension = points.get(0).length;
        this.points = points;
        this.centroid = new double[this.dimension];
        this.updateCentroid();  
    }

    public void addPoint(double[] point){
        if (point.length != this.dimension) throw new Error("Point dimension mismatch");
        if (this.points.contains(point)) return;
        this.points.add(point);
    }

    public double[] getCentroid(){
        return this.centroid;
    }

    public void updateCentroid(){
        double[] acc = new double[this.dimension];
        int pointsSize = this.points.size();
        if (pointsSize == 0) return;
        
        for (double[] point : this.points) {
            for (int i = 0; i < this.dimension; i++) {
                acc[i] += point[i];
            }
        }
        for (int i = 0; i < this.dimension; i++) {
            this.centroid[i] = acc[i] / pointsSize;
        }
    }
    
    public void clearPoints(){
        this.points.clear();
    }   
}

## DrawListener

In [6]:
// DrawListener.java
public interface DrawListener {

    /**
     * Invoked when the mouse has been pressed.
     *
     * @param x the x-coordinate of the mouse
     * @param y the y-coordinate of the mouse
     */
    void mousePressed(double x, double y);

    /**
     * Invoked when the mouse has been dragged.
     *
     * @param x the x-coordinate of the mouse
     * @param y the y-coordinate of the mouse
     */
    void mouseDragged(double x, double y);

    /**
     * Invoked when the mouse has been released.
     *
     * @param x the x-coordinate of the mouse
     * @param y the y-coordinate of the mouse
     */
    void mouseReleased(double x, double y);

    /**
     * Invoked when the mouse has been clicked (pressed and released).
     *
     * @param x the x-coordinate of the mouse
     * @param y the y-coordinate of the mouse
     */
    void mouseClicked(double x, double y);

    /**
     * Invoked when a key has been typed.
     *
     * @param c the character typed
     */
    void keyTyped(char c);

    /**
     * Invoked when a key has been pressed.
     *
     * @param keycode the key combination pressed
     */
    void keyPressed(int keycode);

    /**
     * Invoked when a key has been released.
     *
     * @param keycode the key combination released
     */
    void keyReleased(int keycode);
}

## Draw

In [7]:
// Draw.java
public final class Draw implements ActionListener, MouseListener, MouseMotionListener, KeyListener {

    /**
     *  The color black.
     */
    public static final Color BLACK = Color.BLACK;

    /**
     *  The color blue.
     */
    public static final Color BLUE = Color.BLUE;

    /**
     *  The color cyan.
     */
    public static final Color CYAN = Color.CYAN;

    /**
     *  The color dark gray.
     */
    public static final Color DARK_GRAY = Color.DARK_GRAY;

    /**
     *  The color gray.
     */
    public static final Color GRAY = Color.GRAY;

    /**
     *  The color green.
     */
    public static final Color GREEN  = Color.GREEN;

    /**
     *  The color light gray.
     */
    public static final Color LIGHT_GRAY = Color.LIGHT_GRAY;

    /**
     *  The color magenta.
     */
    public static final Color MAGENTA = Color.MAGENTA;

    /**
     *  The color orange.
     */
    public static final Color ORANGE = Color.ORANGE;

    /**
     *  The color pink.
     */
    public static final Color PINK = Color.PINK;

    /**
     *  The color red.
     */
    public static final Color RED = Color.RED;

    /**
     *  The color white.
     */
    public static final Color WHITE = Color.WHITE;

    /**
     *  The color yellow.
     */
    public static final Color YELLOW = Color.YELLOW;

    /**
     * Shade of blue used in Introduction to Programming in Java.
     * It is Pantone 300U. The RGB values are approximately (9, 90, 166).
     */
    public static final Color BOOK_BLUE = new Color(9, 90, 166);

    /**
     * Shade of light blue used in Introduction to Programming in Java.
     * The RGB values are approximately (103, 198, 243).
     */
    public static final Color BOOK_LIGHT_BLUE = new Color(103, 198, 243);
    
    /**
     * Shade of red used in <em>Algorithms, 4th edition</em>.
     * It is Pantone 1805U. The RGB values are approximately (150, 35, 31).
     */
    public static final Color BOOK_RED = new Color(150, 35, 31);

    /**
     * Shade of orange used in Princeton's identity.
     * It is PMS 158. The RGB values are approximately (245, 128, 37).
     */
    public static final Color PRINCETON_ORANGE = new Color(245, 128, 37);

    // default colors
    private static final Color DEFAULT_PEN_COLOR   = BLACK;
    private static final Color DEFAULT_CLEAR_COLOR = WHITE;

    // boundary of drawing canvas, 0% border
    private static final double BORDER = 0.0;
    private static final double DEFAULT_XMIN = 0.0;
    private static final double DEFAULT_XMAX = 1.0;
    private static final double DEFAULT_YMIN = 0.0;
    private static final double DEFAULT_YMAX = 1.0;

    // default canvas size is SIZE-by-SIZE
    private static final int DEFAULT_SIZE = 512;

    // default pen radius
    private static final double DEFAULT_PEN_RADIUS = 0.002;

    // default font
    private static final Font DEFAULT_FONT = new Font("SansSerif", Font.PLAIN, 16);

    // current pen color
    private Color penColor;

    // canvas size
    private int width  = DEFAULT_SIZE;
    private int height = DEFAULT_SIZE;

    // current pen radius
    private double penRadius;

    // show we draw immediately or wait until next show?
    private boolean defer = false;

    private double xmin, ymin, xmax, ymax;

    // name of window
    private String name = "Draw";

    // for synchronization
    private final Object mouseLock = new Object();
    private final Object keyLock = new Object();

    // current font
    private Font font;

    // the JLabel for drawing
    private JLabel draw;

    // double buffered graphics
    private BufferedImage offscreenImage, onscreenImage;
    private Graphics2D offscreen, onscreen;

    // the frame for drawing to the screen
    private JFrame frame = new JFrame();

    // mouse state
    private boolean isMousePressed = false;
    private double mouseX = 0;
    private double mouseY = 0;

    // keyboard state
    private final LinkedList<Character> keysTyped = new LinkedList<Character>();
    private final TreeSet<Integer> keysDown = new TreeSet<Integer>();

    // event-based listeners
    private final ArrayList<DrawListener> listeners = new ArrayList<DrawListener>();


    /**
     * Initializes an empty drawing object with the given name.
     *
     * @param name the title of the drawing window.
     */
    public Draw(String name) {
        this.name = name;
        init();
    }

    /**
     * Initializes an empty drawing object.
     */
    public Draw() {
        init();
    }

    private void init() {
        offscreenImage = new BufferedImage(2*width, 2*height, BufferedImage.TYPE_INT_ARGB);
        onscreenImage  = new BufferedImage(2*width, 2*height, BufferedImage.TYPE_INT_ARGB);
        offscreen = offscreenImage.createGraphics();
        onscreen  = onscreenImage.createGraphics();
        offscreen.scale(2.0, 2.0);  // since we made it 2x as big

        setXscale();
        setYscale();
        offscreen.setColor(DEFAULT_CLEAR_COLOR);
        offscreen.fillRect(0, 0, width, height);
        setPenColor();
        setPenRadius();
        setFont();
        clear();

        // add antialiasing
        RenderingHints hints = new RenderingHints(RenderingHints.KEY_ANTIALIASING,
                                                  RenderingHints.VALUE_ANTIALIAS_ON);
        hints.put(RenderingHints.KEY_RENDERING, RenderingHints.VALUE_RENDER_QUALITY);
        offscreen.addRenderingHints(hints);

        // frame stuff
        RetinaImageIcon icon = new RetinaImageIcon(onscreenImage);
        draw = new JLabel(icon);

        draw.addMouseListener(this);
        draw.addMouseMotionListener(this);

        frame.setContentPane(draw);
        frame.addKeyListener(this);    // JLabel cannot get keyboard focus
        frame.setResizable(false);
        frame.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);            // closes all windows
        // frame.setDefaultCloseOperation(JFrame.DISPOSE_ON_CLOSE);      // closes only current window
        frame.setFocusTraversalKeysEnabled(false);  // to recognize VK_TAB with isKeyPressed()
        frame.setTitle(name);
        frame.setJMenuBar(createMenuBar());
        frame.pack();
        frame.requestFocusInWindow();
        frame.setVisible(true);
    }


    /**
     * Sets the upper-left hand corner of the drawing window to be (x, y), where (0, 0) is upper left.
     *
     * @param  x the number of pixels from the left
     * @param  y the number of pixels from the top
     * @throws IllegalArgumentException if the width or height is 0 or negative
     */
    public void setLocationOnScreen(int x, int y) {
        if (x <= 0 || y <= 0) throw new IllegalArgumentException();
        frame.setLocation(x, y);
    }

    /**
     * Sets the default close operation.
     *
     * @param  value the value, typically {@code JFrame.EXIT_ON_CLOSE}
     *         (close all windows) or {@code JFrame.DISPOSE_ON_CLOSE}
     *         (close current window)
     */
    public void setDefaultCloseOperation(int value) {
        frame.setDefaultCloseOperation(value);
    }
       

    /**
     * Sets the canvas (drawing area) to be <em>width</em>-by-<em>height</em> pixels.
     * This also erases the current drawing and resets the coordinate system, pen radius,
     * pen color, and font back to their default values.
     * Ordinarly, this method is called once, at the very beginning of a program.
     *
     * @param  canvasWidth the width as a number of pixels
     * @param  canvasHeight the height as a number of pixels
     * @throws IllegalArgumentException unless both {@code canvasWidth}
     *         and {@code canvasHeight} are positive
     */
    public void setCanvasSize(int canvasWidth, int canvasHeight) {
        if (canvasWidth < 1 || canvasHeight < 1) {
            throw new IllegalArgumentException("width and height must be positive");
        }
        width = canvasWidth;
        height = canvasHeight;
        init();
    }


    // create the menu bar (changed to private)
    private JMenuBar createMenuBar() {
        JMenuBar menuBar = new JMenuBar();
        JMenu menu = new JMenu("File");
        menuBar.add(menu);
        JMenuItem menuItem1 = new JMenuItem(" Save...   ");
        menuItem1.addActionListener(this);
        // Java 10+: replace getMenuShortcutKeyMask() with getMenuShortcutKeyMaskEx()
        menuItem1.setAccelerator(KeyStroke.getKeyStroke(KeyEvent.VK_S,
                                Toolkit.getDefaultToolkit().getMenuShortcutKeyMaskEx()));
        menu.add(menuItem1);
        return menuBar;
    }


   /***************************************************************************
    *  User and screen coordinate systems.
    ***************************************************************************/

    // throw an IllegalArgumentException if x is NaN or infinite
    private static void validate(double x, String name) {
        if (Double.isNaN(x)) throw new IllegalArgumentException(name + " is NaN");
        if (Double.isInfinite(x)) throw new IllegalArgumentException(name + " is infinite");
    }

    // throw an IllegalArgumentException if s is null
    private static void validateNonnegative(double x, String name) {
        if (x < 0) throw new IllegalArgumentException(name + " negative");
    }

    // throw an IllegalArgumentException if s is null
    private static void validateNotNull(Object x, String name) {
        if (x == null) throw new IllegalArgumentException(name + " is null");
    }

    /**
     * Sets the x-scale to be the default (between 0.0 and 1.0).
     */
    public void setXscale() {
        setXscale(DEFAULT_XMIN, DEFAULT_XMAX);
    }

    /**
     * Sets the y-scale to be the default (between 0.0 and 1.0).
     */
    public void setYscale() {
        setYscale(DEFAULT_YMIN, DEFAULT_YMAX);
    }

    /**
     * Sets the x-scale.
     *
     * @param min the minimum value of the x-scale
     * @param max the maximum value of the x-scale
     * @throws IllegalArgumentException if {@code (max == min)}
     * @throws IllegalArgumentException if either {@code min} or {@code max} is either NaN or infinite
     */
    public void setXscale(double min, double max) {
        validate(min, "min");
        validate(max, "max");
        double size = max - min;
        if (size == 0.0) throw new IllegalArgumentException("the min and max are the same");
        xmin = min - BORDER * size;
        xmax = max + BORDER * size;
    }

    /**
     * Sets the y-scale.
     *
     * @param min the minimum value of the y-scale
     * @param max the maximum value of the y-scale
     * @throws IllegalArgumentException if {@code (max == min)}
     * @throws IllegalArgumentException if either {@code min} or {@code max} is either NaN or infinite
     */
    public void setYscale(double min, double max) {
        validate(min, "min");
        validate(max, "max");
        double size = max - min;
        if (size == 0.0) throw new IllegalArgumentException("the min and max are the same");
        ymin = min - BORDER * size;
        ymax = max + BORDER * size;
    }

    // helper functions that scale from user coordinates to screen coordinates and back
    private double  scaleX(double x) { return width  * (x - xmin) / (xmax - xmin); }
    private double  scaleY(double y) { return height * (ymax - y) / (ymax - ymin); }
    private double factorX(double w) { return w * width  / Math.abs(xmax - xmin);  }
    private double factorY(double h) { return h * height / Math.abs(ymax - ymin);  }
    private double   userX(double x) { return xmin + x * (xmax - xmin) / width;    }
    private double   userY(double y) { return ymax - y * (ymax - ymin) / height;   }


    /**
     * Clears the screen to the default color (white).
     */
    public void clear() {
        clear(DEFAULT_CLEAR_COLOR);
    }

    /**
     * Clears the screen to the given color.
     *
     * @param color the color to make the background
     * @throws IllegalArgumentException if {@code color} is {@code null}
     */
    public void clear(Color color) {
        validateNotNull(color, "color");
        offscreen.setColor(color);
        offscreen.fillRect(0, 0, width, height);
        offscreen.setColor(penColor);
        draw();
    }

    /**
     * Gets the current pen radius.
     *
     * @return the current pen radius
     */
    public double getPenRadius() {
        return penRadius;
    }

    /**
     * Sets the pen size to the default (.002).
     */
    public void setPenRadius() {
        setPenRadius(DEFAULT_PEN_RADIUS);
    }

    /**
     * Sets the radius of the pen to the given size.
     *
     * @param  radius the radius of the pen
     * @throws IllegalArgumentException if {@code radius} is negative, NaN, or infinite
     */
    public void setPenRadius(double radius) {
        validate(radius, "pen radius");
        validateNonnegative(radius, "pen radius");

        penRadius = radius * DEFAULT_SIZE;
        BasicStroke stroke = new BasicStroke((float) penRadius, BasicStroke.CAP_ROUND, BasicStroke.JOIN_ROUND);
        // BasicStroke stroke = new BasicStroke((float) penRadius);
        offscreen.setStroke(stroke);
    }

    /**
     * Gets the current pen color.
     *
     * @return the current pen color
     */
    public Color getPenColor() {
        return penColor;
    }

    /**
     * Sets the pen color to the default color (black).
     */
    public void setPenColor() {
        setPenColor(DEFAULT_PEN_COLOR);
    }

    /**
     * Sets the pen color to the given color.
     *
     * @param color the color to make the pen
     * @throws IllegalArgumentException if {@code color} is {@code null}
     */
    public void setPenColor(Color color) {
        validateNotNull(color, "color");
        penColor = color;
        offscreen.setColor(penColor);
    }

    /**
     * Sets the pen color to the given RGB color.
     *
     * @param  red the amount of red (between 0 and 255)
     * @param  green the amount of green (between 0 and 255)
     * @param  blue the amount of blue (between 0 and 255)
     * @throws IllegalArgumentException if {@code red}, {@code green},
     *         or {@code blue} is outside its prescribed range
     */
    public void setPenColor(int red, int green, int blue) {
        if (red   < 0 || red   >= 256) throw new IllegalArgumentException("red must be between 0 and 255");
        if (green < 0 || green >= 256) throw new IllegalArgumentException("green must be between 0 and 255");
        if (blue  < 0 || blue  >= 256) throw new IllegalArgumentException("blue must be between 0 and 255");
        setPenColor(new Color(red, green, blue));
    }


    /**
     * Turns on xor mode.
     */
    public void xorOn() {
        offscreen.setXORMode(DEFAULT_CLEAR_COLOR);
    }

    /**
     * Turns off xor mode.
     */
    public void xorOff() {
        offscreen.setPaintMode();
    }

    /**
     * Gets the current {@code JLabel} for use in some other GUI.
     *
     * @return the current {@code JLabel}
     */
    public JLabel getJLabel() {
        return draw;
    }

    /**
     * Gets the current font.
     *
     * @return the current font
     */
    public Font getFont() {
        return font;
    }

    /**
     * Sets the font to the default font (sans serif, 16 point).
     */
    public void setFont() {
        setFont(DEFAULT_FONT);
    }

    /**
     * Sets the font to the given value.
     *
     * @param font the font
     * @throws IllegalArgumentException if {@code font} is {@code null}
     */
    public void setFont(Font font) {
        validateNotNull(font, "font");
        this.font = font;
    }


   /***************************************************************************
    *  Drawing geometric shapes.
    ***************************************************************************/

    /**
     * Draws a line from (x0, y0) to (x1, y1).
     *
     * @param x0 the x-coordinate of the starting point
     * @param y0 the y-coordinate of the starting point
     * @param x1 the x-coordinate of the destination point
     * @param y1 the y-coordinate of the destination point
     * @throws IllegalArgumentException if any coordinate is either NaN or infinite
     */
    public void line(double x0, double y0, double x1, double y1) {
        validate(x0, "x0");
        validate(y0, "y0");
        validate(x1, "x1");
        validate(y1, "y1");
        offscreen.draw(new Line2D.Double(scaleX(x0), scaleY(y0), scaleX(x1), scaleY(y1)));
        draw();
    }

    /**
     * Draws one pixel at (x, y).
     *
     * @param x the x-coordinate of the pixel
     * @param y the y-coordinate of the pixel
     * @throws IllegalArgumentException if {@code x} or {@code y} is either NaN or infinite
     */
    private void pixel(double x, double y) {
        validate(x, "x");
        validate(y, "y");
        offscreen.fillRect((int) Math.round(scaleX(x)), (int) Math.round(scaleY(y)), 1, 1);
    }

    /**
     * Draws a point at (x, y).
     *
     * @param x the x-coordinate of the point
     * @param y the y-coordinate of the point
     * @throws IllegalArgumentException if either {@code x} or {@code y} is either NaN or infinite
     */
    public void point(double x, double y) {
        validate(x, "x");
        validate(y, "y");

        double xs = scaleX(x);
        double ys = scaleY(y);
        double r = penRadius;
        // double ws = factorX(2*r);
        // double hs = factorY(2*r);
        // if (ws <= 1 && hs <= 1) pixel(x, y);
        if (r <= 1) pixel(x, y);
        else offscreen.fill(new Ellipse2D.Double(xs - r/2, ys - r/2, r, r));
        draw();
    }

    /**
     * Draws a circle of the specified radius, centered at (<em>x</em>, <em>y</em>).
     *
     * @param  x the x-coordinate of the center of the circle
     * @param  y the y-coordinate of the center of the circle
     * @param  radius the radius of the circle
     * @throws IllegalArgumentException if {@code radius} is negative
     * @throws IllegalArgumentException if any argument is either NaN or infinite
     */
    public void circle(double x, double y, double radius) {
        validate(x, "x");
        validate(y, "y");
        validate(radius, "radius");
        validateNonnegative(radius, "radius");

        double xs = scaleX(x);
        double ys = scaleY(y);
        double ws = factorX(2*radius);
        double hs = factorY(2*radius);
        if (ws <= 1 && hs <= 1) pixel(x, y);
        else offscreen.draw(new Ellipse2D.Double(xs - ws/2, ys - hs/2, ws, hs));
        draw();
    }

    /**
     * Draws a filled circle of the specified radius, centered at (<em>x</em>, <em>y</em>).
     *
     * @param  x the x-coordinate of the center of the circle
     * @param  y the y-coordinate of the center of the circle
     * @param  radius the radius of the circle
     * @throws IllegalArgumentException if {@code radius} is negative
     * @throws IllegalArgumentException if any argument is either NaN or infinite
     */
    public void filledCircle(double x, double y, double radius) {
        validate(x, "x");
        validate(y, "y");
        validate(radius, "radius");
        validateNonnegative(radius, "radius");

        double xs = scaleX(x);
        double ys = scaleY(y);
        double ws = factorX(2*radius);
        double hs = factorY(2*radius);
        if (ws <= 1 && hs <= 1) pixel(x, y);
        else offscreen.fill(new Ellipse2D.Double(xs - ws/2, ys - hs/2, ws, hs));
        draw();
    }


    /**
     * Draws an ellipse with the specified semimajor and semiminor axes,
     * centered at (<em>x</em>, <em>y</em>).
     *
     * @param  x the <em>x</em>-coordinate of the center of the ellipse
     * @param  y the <em>y</em>-coordinate of the center of the ellipse
     * @param  semiMajorAxis is the semimajor axis of the ellipse
     * @param  semiMinorAxis is the semiminor axis of the ellipse
     * @throws IllegalArgumentException if either {@code semiMajorAxis}
     *         or {@code semiMinorAxis} is negative
     * @throws IllegalArgumentException if any argument is either NaN or infinite
     */
    public void ellipse(double x, double y, double semiMajorAxis, double semiMinorAxis) {
        validate(x, "x");
        validate(y, "y");
        validate(semiMajorAxis, "semimajor axis");
        validate(semiMinorAxis, "semiminor axis");
        validateNonnegative(semiMajorAxis, "semimajor axis");
        validateNonnegative(semiMinorAxis, "semiminor axis");

        double xs = scaleX(x);
        double ys = scaleY(y);
        double ws = factorX(2*semiMajorAxis);
        double hs = factorY(2*semiMinorAxis);
        if (ws <= 1 && hs <= 1) pixel(x, y);
        else offscreen.draw(new Ellipse2D.Double(xs - ws/2, ys - hs/2, ws, hs));
        draw();
    }

    /**
     * Draws a filled ellipse with the specified semimajor and semiminor axes,
     * centered at (<em>x</em>, <em>y</em>).
     *
     * @param  x the <em>x</em>-coordinate of the center of the ellipse
     * @param  y the <em>y</em>-coordinate of the center of the ellipse
     * @param  semiMajorAxis is the semimajor axis of the ellipse
     * @param  semiMinorAxis is the semiminor axis of the ellipse
     * @throws IllegalArgumentException if either {@code semiMajorAxis}
     *         or {@code semiMinorAxis} is negative
     * @throws IllegalArgumentException if any argument is either NaN or infinite
     */
    public void filledEllipse(double x, double y, double semiMajorAxis, double semiMinorAxis) {
        validate(x, "x");
        validate(y, "y");
        validate(semiMajorAxis, "semimajor axis");
        validate(semiMinorAxis, "semiminor axis");
        validateNonnegative(semiMajorAxis, "semimajor axis");
        validateNonnegative(semiMinorAxis, "semiminor axis");

        double xs = scaleX(x);
        double ys = scaleY(y);
        double ws = factorX(2*semiMajorAxis);
        double hs = factorY(2*semiMinorAxis);
        if (ws <= 1 && hs <= 1) pixel(x, y);
        else offscreen.fill(new Ellipse2D.Double(xs - ws/2, ys - hs/2, ws, hs));
        draw();
    }

    /**
     * Draws a circular arc of the specified radius,
     * centered at (<em>x</em>, <em>y</em>), from angle1 to angle2 (in degrees).
     *
     * @param  x the <em>x</em>-coordinate of the center of the circle
     * @param  y the <em>y</em>-coordinate of the center of the circle
     * @param  radius the radius of the circle
     * @param  angle1 the starting angle. 0 would mean an arc beginning at 3 o'clock.
     * @param  angle2 the angle at the end of the arc. For example, if
     *         you want a 90 degree arc, then angle2 should be angle1 + 90.
     * @throws IllegalArgumentException if {@code radius} is negative
     * @throws IllegalArgumentException if any argument is either NaN or infinite
     */
    public void arc(double x, double y, double radius, double angle1, double angle2) {
        validate(x, "x");
        validate(y, "y");
        validate(radius, "arc radius");
        validate(angle1, "angle1");
        validate(angle2, "angle2");
        validateNonnegative(radius, "arc radius");

        while (angle2 < angle1) angle2 += 360;
        double xs = scaleX(x);
        double ys = scaleY(y);
        double ws = factorX(2*radius);
        double hs = factorY(2*radius);
        if (ws <= 1 && hs <= 1) pixel(x, y);
        else offscreen.draw(new Arc2D.Double(xs - ws/2, ys - hs/2, ws, hs, angle1, angle2 - angle1, Arc2D.OPEN));
        draw();
    }

    /**
     * Draws a square of the specified size, centered at (<em>x</em>, <em>y</em>).
     *
     * @param  x the <em>x</em>-coordinate of the center of the square
     * @param  y the <em>y</em>-coordinate of the center of the square
     * @param  halfLength one half the length of any side of the square
     * @throws IllegalArgumentException if {@code halfLength} is negative
     * @throws IllegalArgumentException if any argument is either NaN or infinite
     */
    public void square(double x, double y, double halfLength) {
        validate(x, "x");
        validate(y, "y");
        validate(halfLength, "halfLength");
        validateNonnegative(halfLength, "half length");

        double xs = scaleX(x);
        double ys = scaleY(y);
        double ws = factorX(2*halfLength);
        double hs = factorY(2*halfLength);
        if (ws <= 1 && hs <= 1) pixel(x, y);
        else offscreen.draw(new Rectangle2D.Double(xs - ws/2, ys - hs/2, ws, hs));
        draw();
    }

    /**
     * Draws a square of the specified size, centered at (<em>x</em>, <em>y</em>).
     *
     * @param  x the <em>x</em>-coordinate of the center of the square
     * @param  y the <em>y</em>-coordinate of the center of the square
     * @param  halfLength one half the length of any side of the square
     * @throws IllegalArgumentException if {@code halfLength} is negative
     * @throws IllegalArgumentException if any argument is either NaN or infinite
     */
    public void filledSquare(double x, double y, double halfLength) {
        validate(x, "x");
        validate(y, "y");
        validate(halfLength, "halfLength");
        validateNonnegative(halfLength, "half length");

        double xs = scaleX(x);
        double ys = scaleY(y);
        double ws = factorX(2*halfLength);
        double hs = factorY(2*halfLength);
        if (ws <= 1 && hs <= 1) pixel(x, y);
        else offscreen.fill(new Rectangle2D.Double(xs - ws/2, ys - hs/2, ws, hs));
        draw();
    }


    /**
     * Draws a rectangle of the specified size, centered at (<em>x</em>, <em>y</em>).
     *
     * @param  x the <em>x</em>-coordinate of the center of the rectangle
     * @param  y the <em>y</em>-coordinate of the center of the rectangle
     * @param  halfWidth one half the width of the rectangle
     * @param  halfHeight one half the height of the rectangle
     * @throws IllegalArgumentException if either {@code halfWidth} or {@code halfHeight} is negative
     * @throws IllegalArgumentException if any argument is either NaN or infinite
     */
    public void rectangle(double x, double y, double halfWidth, double halfHeight) {
        validate(x, "x");
        validate(y, "y");
        validate(halfWidth, "halfWidth");
        validate(halfHeight, "halfHeight");
        validateNonnegative(halfWidth, "half width");
        validateNonnegative(halfHeight, "half height");

        double xs = scaleX(x);
        double ys = scaleY(y);
        double ws = factorX(2*halfWidth);
        double hs = factorY(2*halfHeight);
        if (ws <= 1 && hs <= 1) pixel(x, y);
        else offscreen.draw(new Rectangle2D.Double(xs - ws/2, ys - hs/2, ws, hs));
        draw();
    }

    /**
     * Draws a filled rectangle of the specified size, centered at (<em>x</em>, <em>y</em>).
     *
     * @param  x the <em>x</em>-coordinate of the center of the rectangle
     * @param  y the <em>y</em>-coordinate of the center of the rectangle
     * @param  halfWidth one half the width of the rectangle
     * @param  halfHeight one half the height of the rectangle
     * @throws IllegalArgumentException if either {@code halfWidth} or {@code halfHeight} is negative
     * @throws IllegalArgumentException if any argument is either NaN or infinite
     */
    public void filledRectangle(double x, double y, double halfWidth, double halfHeight) {
        validate(x, "x");
        validate(y, "y");
        validate(halfWidth, "halfWidth");
        validate(halfHeight, "halfHeight");
        validateNonnegative(halfWidth, "half width");
        validateNonnegative(halfHeight, "half height");

        double xs = scaleX(x);
        double ys = scaleY(y);
        double ws = factorX(2*halfWidth);
        double hs = factorY(2*halfHeight);
        if (ws <= 1 && hs <= 1) pixel(x, y);
        else offscreen.fill(new Rectangle2D.Double(xs - ws/2, ys - hs/2, ws, hs));
        draw();
    }

    /**
     * Draws a polygon with the vertices 
     * (<em>x</em><sub>0</sub>, <em>y</em><sub>0</sub>),
     * (<em>x</em><sub>1</sub>, <em>y</em><sub>1</sub>), ...,
     * (<em>x</em><sub><em>n</em>–1</sub>, <em>y</em><sub><em>n</em>–1</sub>).
     *
     * @param  x an array of all the <em>x</em>-coordinates of the polygon
     * @param  y an array of all the <em>y</em>-coordinates of the polygon
     * @throws IllegalArgumentException unless {@code x[]} and {@code y[]}
     *         are of the same length
     * @throws IllegalArgumentException if any coordinate is either NaN or infinite
     * @throws IllegalArgumentException if either {@code x[]} or {@code y[]} is {@code null}
     */
    public void polygon(double[] x, double[] y) {
        validateNotNull(x, "x-coordinate array");
        validateNotNull(y, "y-coordinate array");
        for (int i = 0; i < x.length; i++) validate(x[i], "x[" + i + "]");
        for (int i = 0; i < y.length; i++) validate(y[i], "y[" + i + "]");

        int n1 = x.length;
        int n2 = y.length;
        if (n1 != n2) throw new IllegalArgumentException("arrays must be of the same length");
        int n = n1;
        if (n == 0) return;

        GeneralPath path = new GeneralPath();
        path.moveTo((float) scaleX(x[0]), (float) scaleY(y[0]));
        for (int i = 0; i < n; i++)
            path.lineTo((float) scaleX(x[i]), (float) scaleY(y[i]));
        path.closePath();
        offscreen.draw(path);
        draw();
    }

    /**
     * Draws a filled polygon with the vertices 
     * (<em>x</em><sub>0</sub>, <em>y</em><sub>0</sub>),
     * (<em>x</em><sub>1</sub>, <em>y</em><sub>1</sub>), ...,
     * (<em>x</em><sub><em>n</em>–1</sub>, <em>y</em><sub><em>n</em>–1</sub>).
     *
     * @param  x an array of all the <em>x</em>-coordinates of the polygon
     * @param  y an array of all the <em>y</em>-coordinates of the polygon
     * @throws IllegalArgumentException unless {@code x[]} and {@code y[]}
     *         are of the same length
     * @throws IllegalArgumentException if any coordinate is either NaN or infinite
     * @throws IllegalArgumentException if either {@code x[]} or {@code y[]} is {@code null}
     */
    public void filledPolygon(double[] x, double[] y) {
        validateNotNull(x, "x-coordinate array");
        validateNotNull(y, "y-coordinate array");
        for (int i = 0; i < x.length; i++) validate(x[i], "x[" + i + "]");
        for (int i = 0; i < y.length; i++) validate(y[i], "y[" + i + "]");

        int n1 = x.length;
        int n2 = y.length;
        if (n1 != n2) throw new IllegalArgumentException("arrays must be of the same length");
        int n = n1;
        if (n == 0) return;

        GeneralPath path = new GeneralPath();
        path.moveTo((float) scaleX(x[0]), (float) scaleY(y[0]));
        for (int i = 0; i < n; i++)
            path.lineTo((float) scaleX(x[i]), (float) scaleY(y[i]));
        path.closePath();
        offscreen.fill(path);
        draw();
    }



   /***************************************************************************
    *  Drawing images.
    ***************************************************************************/

    // get an image from the given filename
    private static Image getImage(String filename) {
        if (filename == null) throw new IllegalArgumentException();

        // to read from file
        ImageIcon icon = new ImageIcon(filename);

        // try to read from URL
        if ((icon == null) || (icon.getImageLoadStatus() != MediaTracker.COMPLETE)) {
            try {
                URL url = new URL(filename);
                icon = new ImageIcon(url);
            }
            catch (MalformedURLException e) {
                /* not a url */
            }
        }

        // in case file is inside a .jar (classpath relative to StdDraw)
        /*if ((icon == null) || (icon.getImageLoadStatus() != MediaTracker.COMPLETE)) {
            URL url = StdDraw.class.getResource(filename);
            if (url != null)
                icon = new ImageIcon(url);
        }*/

        // in case file is inside a .jar (classpath relative to root of jar)
        if ((icon == null) || (icon.getImageLoadStatus() != MediaTracker.COMPLETE)) {
            URL url = Draw.class.getResource("/" + filename);
            if (url == null) throw new IllegalArgumentException("image " + filename + " not found");
            icon = new ImageIcon(url);
        }

        return icon.getImage();
    }

    /**
     * Draws the specified image centered at (<em>x</em>, <em>y</em>).
     * The supported image formats are JPEG, PNG, and GIF.
     * As an optimization, the picture is cached, so there is no performance
     * penalty for redrawing the same image multiple times (e.g., in an animation).
     * However, if you change the picture file after drawing it, subsequent
     * calls will draw the original picture.
     *
     * @param  x the center <em>x</em>-coordinate of the image
     * @param  y the center <em>y</em>-coordinate of the image
     * @param  filename the name of the image/picture, e.g., "ball.gif"
     * @throws IllegalArgumentException if the image filename is invalid
     * @throws IllegalArgumentException if either {@code x} or {@code y} is either NaN or infinite
     */
    public void picture(double x, double y, String filename) {
        validate(x, "x");
        validate(y, "y");
        validateNotNull(filename, "filename");

        Image image = getImage(filename);
        double xs = scaleX(x);
        double ys = scaleY(y);
        int ws = image.getWidth(null);
        int hs = image.getHeight(null);
        if (ws < 0 || hs < 0) throw new IllegalArgumentException("image " + filename + " is corrupt");

        offscreen.drawImage(image, (int) Math.round(xs - ws/2.0), (int) Math.round(ys - hs/2.0), null);
        draw();
    }

    /**
     * Draws the specified image centered at (<em>x</em>, <em>y</em>),
     * rotated given number of degrees.
     * The supported image formats are JPEG, PNG, and GIF.
     *
     * @param  x the center <em>x</em>-coordinate of the image
     * @param  y the center <em>y</em>-coordinate of the image
     * @param  filename the name of the image/picture, e.g., "ball.gif"
     * @param  degrees is the number of degrees to rotate counterclockwise
     * @throws IllegalArgumentException if the image filename is invalid
     * @throws IllegalArgumentException if {@code x}, {@code y}, {@code degrees} is NaN or infinite
     * @throws IllegalArgumentException if {@code filename} is {@code null}
     */
    public void picture(double x, double y, String filename, double degrees) {
        validate(x, "x");
        validate(y, "y");
        validate(degrees, "degrees");
        validateNotNull(filename, "filename");

        Image image = getImage(filename);
        double xs = scaleX(x);
        double ys = scaleY(y);
        int ws = image.getWidth(null);
        int hs = image.getHeight(null);
        if (ws < 0 || hs < 0) throw new IllegalArgumentException("image " + filename + " is corrupt");

        offscreen.rotate(Math.toRadians(-degrees), xs, ys);
        offscreen.drawImage(image, (int) Math.round(xs - ws/2.0), (int) Math.round(ys - hs/2.0), null);
        offscreen.rotate(Math.toRadians(+degrees), xs, ys);

        draw();
    }

    /**
     * Draws the specified image centered at (<em>x</em>, <em>y</em>),
     * rescaled to the specified bounding box.
     * The supported image formats are JPEG, PNG, and GIF.
     *
     * @param  x the center <em>x</em>-coordinate of the image
     * @param  y the center <em>y</em>-coordinate of the image
     * @param  filename the name of the image/picture, e.g., "ball.gif"
     * @param  scaledWidth the width of the scaled image (in screen coordinates)
     * @param  scaledHeight the height of the scaled image (in screen coordinates)
     * @throws IllegalArgumentException if either {@code scaledWidth}
     *         or {@code scaledHeight} is negative
     * @throws IllegalArgumentException if the image filename is invalid
     * @throws IllegalArgumentException if {@code x} or {@code y} is either NaN or infinite
     * @throws IllegalArgumentException if {@code filename} is {@code null}
     */
    public void picture(double x, double y, String filename, double scaledWidth, double scaledHeight) {
        validate(x, "x");
        validate(y, "y");
        validate(scaledWidth, "scaled width");
        validate(scaledHeight, "scaled height");
        validateNotNull(filename, "filename");
        validateNonnegative(scaledWidth, "scaled width");
        validateNonnegative(scaledHeight, "scaled height");

        Image image = getImage(filename);
        double xs = scaleX(x);
        double ys = scaleY(y);
        double ws = factorX(scaledWidth);
        double hs = factorY(scaledHeight);
        if (ws < 0 || hs < 0) throw new IllegalArgumentException("image " + filename + " is corrupt");
        if (ws <= 1 && hs <= 1) pixel(x, y);
        else {
            offscreen.drawImage(image, (int) Math.round(xs - ws/2.0),
                                       (int) Math.round(ys - hs/2.0),
                                       (int) Math.round(ws),
                                       (int) Math.round(hs), null);
        }
        draw();
    }


    /**
     * Draws the specified image centered at (<em>x</em>, <em>y</em>), rotated
     * given number of degrees, and rescaled to the specified bounding box.
     * The supported image formats are JPEG, PNG, and GIF.
     *
     * @param  x the center <em>x</em>-coordinate of the image
     * @param  y the center <em>y</em>-coordinate of the image
     * @param  filename the name of the image/picture, e.g., "ball.gif"
     * @param  scaledWidth the width of the scaled image (in screen coordinates)
     * @param  scaledHeight the height of the scaled image (in screen coordinates)
     * @param  degrees is the number of degrees to rotate counterclockwise
     * @throws IllegalArgumentException if either {@code scaledWidth}
     *         or {@code scaledHeight} is negative
     * @throws IllegalArgumentException if the image filename is invalid
     */
    public void picture(double x, double y, String filename, double scaledWidth, double scaledHeight, double degrees) {
        validate(x, "x");
        validate(y, "y");
        validate(scaledWidth, "scaled width");
        validate(scaledHeight, "scaled height");
        validate(degrees, "degrees");
        validateNotNull(filename, "filename");
        validateNonnegative(scaledWidth, "scaled width");
        validateNonnegative(scaledHeight, "scaled height");

        Image image = getImage(filename);
        double xs = scaleX(x);
        double ys = scaleY(y);
        double ws = factorX(scaledWidth);
        double hs = factorY(scaledHeight);
        if (ws < 0 || hs < 0) throw new IllegalArgumentException("image " + filename + " is corrupt");
        if (ws <= 1 && hs <= 1) pixel(x, y);

        offscreen.rotate(Math.toRadians(-degrees), xs, ys);
        offscreen.drawImage(image, (int) Math.round(xs - ws/2.0),
                                   (int) Math.round(ys - hs/2.0),
                                   (int) Math.round(ws),
                                   (int) Math.round(hs), null);
        offscreen.rotate(Math.toRadians(+degrees), xs, ys);

        draw();
    }


   /***************************************************************************
    *  Drawing text.
    ***************************************************************************/

    /**
     * Writes the given text string in the current font, centered at (<em>x</em>, <em>y</em>).
     *
     * @param  x the center <em>x</em>-coordinate of the text
     * @param  y the center <em>y</em>-coordinate of the text
     * @param  text the text to write
     * @throws IllegalArgumentException if {@code text} is {@code null}
     * @throws IllegalArgumentException if {@code x} or {@code y} is either NaN or infinite
     */
    public void text(double x, double y, String text) {
        validate(x, "x");
        validate(y, "y");
        validateNotNull(text, "text");

        offscreen.setFont(font);
        FontMetrics metrics = offscreen.getFontMetrics();
        double xs = scaleX(x);
        double ys = scaleY(y);
        int ws = metrics.stringWidth(text);
        int hs = metrics.getDescent();
        offscreen.drawString(text, (float) (xs - ws/2.0), (float) (ys + hs));
        draw();
    }

    /**
     * Writes the given text string in the current font, centered at (<em>x</em>, <em>y</em>) and
     * rotated by the specified number of degrees.
     * @param  x the center <em>x</em>-coordinate of the text
     * @param  y the center <em>y</em>-coordinate of the text
     * @param  text the text to write
     * @param  degrees is the number of degrees to rotate counterclockwise
     * @throws IllegalArgumentException if {@code text} is {@code null}
     * @throws IllegalArgumentException if {@code x}, {@code y}, or {@code degrees} is either NaN or infinite
     */
    public void text(double x, double y, String text, double degrees) {
        validate(x, "x");
        validate(y, "y");
        validate(degrees, "degrees");
        validateNotNull(text, "text");

        double xs = scaleX(x);
        double ys = scaleY(y);
        offscreen.rotate(Math.toRadians(-degrees), xs, ys);
        text(x, y, text);
        offscreen.rotate(Math.toRadians(+degrees), xs, ys);
    }

    /**
     * Writes the given text string in the current font, left-aligned at (<em>x</em>, <em>y</em>).
     * @param  x the <em>x</em>-coordinate of the text
     * @param  y the <em>y</em>-coordinate of the text
     * @param  text the text
     * @throws IllegalArgumentException if {@code text} is {@code null}
     * @throws IllegalArgumentException if {@code x} or {@code y} is either NaN or infinite
     */
    public void textLeft(double x, double y, String text) {
        validate(x, "x");
        validate(y, "y");
        validateNotNull(text, "text");

        offscreen.setFont(font);
        FontMetrics metrics = offscreen.getFontMetrics();
        double xs = scaleX(x);
        double ys = scaleY(y);
        // int ws = metrics.stringWidth(text);
        int hs = metrics.getDescent();
        offscreen.drawString(text, (float) xs, (float) (ys + hs));
        draw();
    }

    /**
     * Writes the given text string in the current font, right-aligned at (<em>x</em>, <em>y</em>).
     *
     * @param  x the <em>x</em>-coordinate of the text
     * @param  y the <em>y</em>-coordinate of the text
     * @param  text the text to write
     * @throws IllegalArgumentException if {@code text} is {@code null}
     * @throws IllegalArgumentException if {@code x} or {@code y} is either NaN or infinite
     */
    public void textRight(double x, double y, String text) {
        validate(x, "x");
        validate(y, "y");
        validateNotNull(text, "text");

        offscreen.setFont(font);
        FontMetrics metrics = offscreen.getFontMetrics();
        double xs = scaleX(x);
        double ys = scaleY(y);
        int ws = metrics.stringWidth(text);
        int hs = metrics.getDescent();
        offscreen.drawString(text, (float) (xs - ws), (float) (ys + hs));
        draw();
    }

    /**
     * Copies the offscreen buffer to the onscreen buffer, pauses for t milliseconds
     * and enables double buffering.
     * @param t number of milliseconds
     * @deprecated replaced by {@link #enableDoubleBuffering()}, {@link #show()}, and {@link #pause(int t)}
     */
    @Deprecated
    public void show(int t) {
        show();
        pause(t);
        enableDoubleBuffering();
    }

    /**
     * Pause for t milliseconds. This method is intended to support computer animations.
     * @param t number of milliseconds
     */
    public void pause(int t) {
        try {
            Thread.sleep(t);
        }
        catch (InterruptedException e) {
            System.out.println("Error sleeping");
        }
    }

    /**
     * Copies offscreen buffer to onscreen buffer. There is no reason to call
     * this method unless double buffering is enabled.
     */
    public void show() {
        onscreen.drawImage(offscreenImage, 0, 0, null);
        frame.repaint();
    }

    // draw onscreen if defer is false
    private void draw() {
        if (!defer) show();
    }

    /**
     * Enable double buffering. All subsequent calls to 
     * drawing methods such as {@code line()}, {@code circle()},
     * and {@code square()} will be deferred until the next call
     * to show(). Useful for animations.
     */
    public void enableDoubleBuffering() {
        defer = true;
    }

    /**
     * Disable double buffering. All subsequent calls to 
     * drawing methods such as {@code line()}, {@code circle()},
     * and {@code square()} will be displayed on screen when called.
     * This is the default.
     */
    public void disableDoubleBuffering() {
        defer = false;
    }

    /**
     * Saves the drawing to using the specified filename.
     * The supported image formats are JPEG and PNG;
     * the filename suffix must be {@code .jpg} or {@code .png}.
     *
     * @param  filename the name of the file with one of the required suffixes
     * @throws IllegalArgumentException if {@code filename} is {@code null}
     */
    public void save(String filename) {
        validateNotNull(filename, "filename");
        File file = new File(filename);
        String suffix = filename.substring(filename.lastIndexOf('.') + 1);

        // png files
        if ("png".equalsIgnoreCase(suffix)) {
            try {
                ImageIO.write(offscreenImage, suffix, file);
            }
            catch (IOException e) {
                e.printStackTrace();
            }
        }

        // need to change from ARGB to RGB for jpeg
        // reference: http://archives.java.sun.com/cgi-bin/wa?A2=ind0404&L=java2d-interest&D=0&P=2727
        else if ("jpg".equalsIgnoreCase(suffix)) {
            WritableRaster raster = offscreenImage.getRaster();
            WritableRaster newRaster;
            newRaster = raster.createWritableChild(0, 0, width, height, 0, 0, new int[] {0, 1, 2});
            DirectColorModel cm = (DirectColorModel) offscreenImage.getColorModel();
            DirectColorModel newCM = new DirectColorModel(cm.getPixelSize(),
                                                          cm.getRedMask(),
                                                          cm.getGreenMask(),
                                                          cm.getBlueMask());
            BufferedImage rgbBuffer = new BufferedImage(newCM, newRaster, false,  null);
            try {
                ImageIO.write(rgbBuffer, suffix, file);
            }
            catch (IOException e) {
                e.printStackTrace();
            }
        }

        else {
            System.out.println("Invalid image file type: " + suffix);
        }
    }


    /**
     * This method cannot be called directly.
     */
    @Override
    public void actionPerformed(ActionEvent e) {
        FileDialog chooser = new FileDialog(frame, "Use a .png or .jpg extension", FileDialog.SAVE);
        chooser.setVisible(true);
        String filename = chooser.getFile();
        if (filename != null) {
            save(chooser.getDirectory() + File.separator + chooser.getFile());
        }
    }



   /***************************************************************************
    *  Event-based interactions.
    ***************************************************************************/

    /**
     * Adds a {@link DrawListener} to listen to keyboard and mouse events.
     *
     * @param listener the {\tt DrawListener} argument
     */
    public void addListener(DrawListener listener) {
        // ensure there is a window for listenting to events
        show();
        listeners.add(listener);
        frame.addKeyListener(this);
        frame.addMouseListener(this);
        frame.addMouseMotionListener(this);
        frame.setFocusable(true); 
    }




   /***************************************************************************
    *  Mouse interactions.
    ***************************************************************************/

    /**
     * Returns true if the mouse is being pressed.
     *
     * @return {@code true} if the mouse is being pressed;
     *         {@code false} otherwise
     */
    public boolean isMousePressed() {
        synchronized (mouseLock) {
            return isMousePressed;
        }
    }

    /**
     * Returns true if the mouse is being pressed.
     *
     * @return {@code true} if the mouse is being pressed;
     *         {@code false} otherwise
     * @deprecated replaced by {@link #isMousePressed()}
     */
    @Deprecated
    public boolean mousePressed() {
        synchronized (mouseLock) {
            return isMousePressed;
        }
    }

    /**
     * Returns the x-coordinate of the mouse.
     * @return the x-coordinate of the mouse
     */
    public double mouseX() {
        synchronized (mouseLock) {
            return mouseX;
        }
    }

    /**
     * Returns the y-coordinate of the mouse.
     *
     * @return the y-coordinate of the mouse
     */
    public double mouseY() {
        synchronized (mouseLock) {
            return mouseY;
        }
    }



    /**
     * This method cannot be called directly.
     */
    @Override
    public void mouseEntered(MouseEvent e) {
        // this body is intentionally left empty
    }

    /**
     * This method cannot be called directly.
     */
    @Override
    public void mouseExited(MouseEvent e) {
        // this body is intentionally left empty
    }

    /**
     * This method cannot be called directly.
     */
    @Override
    public void mousePressed(MouseEvent e) {
        synchronized (mouseLock) {
            mouseX = userX(e.getX());
            mouseY = userY(e.getY());
            isMousePressed = true;
        }
        if (e.getButton() == MouseEvent.BUTTON1) {
            for (DrawListener listener : listeners)
                listener.mousePressed(userX(e.getX()), userY(e.getY()));
        }

    }

    /**
     * This method cannot be called directly.
     */
    @Override
    public void mouseReleased(MouseEvent e) {
        synchronized (mouseLock) {
            isMousePressed = false;
        }
        if (e.getButton() == MouseEvent.BUTTON1) {
            for (DrawListener listener : listeners)
                listener.mouseReleased(userX(e.getX()), userY(e.getY()));
        }
    }

    /**
     * This method cannot be called directly.
     */
    @Override
    public void mouseClicked(MouseEvent e) {
        if (e.getButton() == MouseEvent.BUTTON1) {
            for (DrawListener listener : listeners)
                listener.mouseClicked(userX(e.getX()), userY(e.getY()));
        }
    }


    /**
     * This method cannot be called directly.
     */
    @Override
    public void mouseDragged(MouseEvent e)  {
        synchronized (mouseLock) {
            mouseX = userX(e.getX());
            mouseY = userY(e.getY());
        }
        // doesn't seem to work if a button is specified
        for (DrawListener listener : listeners)
            listener.mouseDragged(userX(e.getX()), userY(e.getY()));
    }

    /**
     * This method cannot be called directly.
     */
    @Override
    public void mouseMoved(MouseEvent e) {
        synchronized (mouseLock) {
            mouseX = userX(e.getX());
            mouseY = userY(e.getY());
        }
    }


   /***************************************************************************
    *  Keyboard interactions.
    ***************************************************************************/

    /**
     * Returns true if the user has typed a key.
     *
     * @return {@code true} if the user has typed a key; {@code false} otherwise
     */
    public boolean hasNextKeyTyped() {
        synchronized (keyLock) {
            return !keysTyped.isEmpty();
        }
    }

    /**
     * The next key typed by the user.
     *
     * @return the next key typed by the user
     */
    public char nextKeyTyped() {
        synchronized (keyLock) {
            return keysTyped.removeLast();
        }
    }

   /**
     * Returns true if the keycode is being pressed.
     * <p>
     * This method takes as an argument the keycode (corresponding to a physical key).
     * It can handle action keys (such as F1 and arrow keys) and modifier keys
     * (such as shift and control).
     * See {@link KeyEvent} for a description of key codes.
     *
     * @param  keycode the keycode to check
     * @return {@code true} if {@code keycode} is currently being pressed;
     *         {@code false} otherwise
     */
    public boolean isKeyPressed(int keycode) {
        synchronized (keyLock) {
            return keysDown.contains(keycode);
        }
    }

    /**
     * This method cannot be called directly.
     */
    @Override
    public void keyTyped(KeyEvent e) {
        synchronized (keyLock) {
            keysTyped.addFirst(e.getKeyChar());
        }

        // notify all listeners
        for (DrawListener listener : listeners)
            listener.keyTyped(e.getKeyChar());
    }

    /**
     * This method cannot be called directly.
     */
    @Override
    public void keyPressed(KeyEvent e) {
        synchronized (keyLock) {
            keysDown.add(e.getKeyCode());
        }

        // notify all listeners
        for (DrawListener listener : listeners)
            listener.keyPressed(e.getKeyCode());
    }

    /**
     * This method cannot be called directly.
     */
    @Override
    public void keyReleased(KeyEvent e) {
        synchronized (keyLock) {
            keysDown.remove(e.getKeyCode());
        }

        // notify all listeners
        for (DrawListener listener : listeners)
            listener.keyReleased(e.getKeyCode());
    }


   /***************************************************************************
    *  For improved resolution on Mac Retina displays.
    ***************************************************************************/

    @SuppressWarnings("serial")
	private static class RetinaImageIcon extends ImageIcon {
    
        public RetinaImageIcon(Image image) {
            super(image);
        }

        public int getIconWidth() {
            return super.getIconWidth() / 2;
        }

        /**
         * Gets the height of the icon.
         *
         * @return the height in pixels of this icon
         */
        public int getIconHeight() {
            return super.getIconHeight() / 2;
        }

        public synchronized void paintIcon(Component c, Graphics g, int x, int y) {
            Graphics2D g2 = (Graphics2D) g.create();
            g2.setRenderingHint(RenderingHints.KEY_INTERPOLATION,RenderingHints.VALUE_INTERPOLATION_BICUBIC);
            g2.setRenderingHint(RenderingHints.KEY_RENDERING,RenderingHints.VALUE_RENDER_QUALITY);
            g2.setRenderingHint(RenderingHints.KEY_ANTIALIASING,RenderingHints.VALUE_ANTIALIAS_ON);
            g2.scale(0.5, 0.5);
            super.paintIcon(c, g2, x * 2, y * 2);
            g2.dispose();
        }
    }




}

In [8]:
/**
 * Test client.
 *
 * @param args the command-line arguments
 */

// create one drawing window
Draw draw1 = new Draw("Test client 1");
draw1.square(0.2, 0.8, 0.1);
draw1.filledSquare(0.8, 0.8, 0.2);
draw1.circle(0.8, 0.2, 0.2);
draw1.setPenColor(Draw.MAGENTA);
draw1.setPenRadius(0.02);
draw1.arc(0.8, 0.2, 0.1, 200, 45);

// create another one
Draw draw2 = new Draw("Test client 2");
draw2.setCanvasSize(900, 200);
// draw a blue diamond
draw2.setPenRadius();
draw2.setPenColor(Draw.BLUE);
double[] x = { 0.1, 0.2, 0.3, 0.2 };
double[] y = { 0.2, 0.3, 0.2, 0.1 };
draw2.filledPolygon(x, y);

// text
draw2.setPenColor(Draw.BLACK);
draw2.text(0.2, 0.5, "bdfdfdfdlack text");
draw2.setPenColor(Draw.WHITE);
draw2.text(0.8, 0.8, "white text");

## InteractiveClosest

In [9]:
// InteractiveClosest.java
public class InteractiveClosest implements DrawListener {

	int nbpoints;
	KDTree tree;
	boolean keytypedbug;

	InteractiveClosest(int nbpoints) {
		this.nbpoints = nbpoints;
		keytypedbug = true;
		inittree();
	}
	

	void inittree() {
		System.out.printf("Initialisation avec %d points.%n", nbpoints);
		tree = null;
		for (int i = 0; i < nbpoints; i++) {
			tree = KDTree.insert(tree, new double[] { Math.random(), Math.random() });
		}
	}

	public void drawTree(KDTree tree, double minx, double maxx, double miny, double maxy) {
		if (tree == null)
			return;

		if (tree.point[0] < minx || tree.point[0] > maxx || tree.point[1] < miny || tree.point[1] > maxy) {
			System.out.println("Error at depth " + tree.depth);
			System.out.println("We should have " + minx + " < point[0] <" + maxx + " and " + miny + " < point[1] < "
					+ maxy + " but");
			if (tree.point[0] < minx)
				System.out.println("point[0] = " + tree.point[0] + " is not > " + minx);
			if (tree.point[0] > maxx)
				System.out.println("point[0] = " + tree.point[0] + " is not < " + maxx);
			if (tree.point[1] < miny)
				System.out.println("point[1] = " + tree.point[1] + " is not > " + miny);
			if (tree.point[1] > maxy)
				System.out.println("point[1] = " + tree.point[1] + " is not < " + maxy);
			draw.setPenColor(Draw.RED);
			draw.filledRectangle((minx+maxx)/2, (miny+maxy)/2, (maxx-minx)/2, (maxy-miny)/2);
			draw.show();
			throw (new RuntimeException("Arbre incorrect. point" + tree.pointToString() + ", depth=" + tree.depth));
		}

		draw.setPenColor(Draw.LIGHT_GRAY);
		if (tree.depth % 2 == 0){
			if (tree.left != null || tree.right != null)
				draw.line(tree.point[0], miny, tree.point[0], maxy);
			drawTree(tree.left, minx, tree.point[0], miny, maxy);
			drawTree(tree.right, tree.point[0], maxx, miny, maxy);
		}
		else {
			if (tree.left != null || tree.right != null)
				draw.line(minx, tree.point[1], maxx, tree.point[1]);
			drawTree(tree.left, minx, maxx, miny, tree.point[1]);
			drawTree(tree.right, minx, maxx, tree.point[1], maxy);
		}
		draw.setPenColor(Draw.BLACK);
		draw.filledCircle(tree.point[0], tree.point[1], .002);

	}

	public void draw() {
		draw.clear();
		drawTree(tree, 0.0, 1.0, 0.0, 1.0);
	}

	@Override
	public void mousePressed(double x, double y) {
		mouseDragged(x, y);
	}

	@Override
	public void mouseDragged(double x, double y) {
		trace.clear();
		champions.clear();
		double[] closest = KDTree.closest(tree, new double[] { x, y });
		// System.out.printf("%f, %f%n", closest[0], closest[1]);
		this.draw();
		draw.setPenColor(Draw.BOOK_LIGHT_BLUE);
		for (double[] c : trace)
			draw.filledCircle(c[0], c[1], .005);

		draw.setPenColor(Draw.PRINCETON_ORANGE);
		champions.addLast(closest);
		double[] prevc = null;
		for (double[] c : champions) {
			draw.filledCircle(c[0], c[1], .005);
			if (prevc != null)
				draw.line(c[0], c[1], prevc[0], prevc[1]);
			prevc = c;
		}

		draw.setPenColor(Draw.BOOK_RED);
		draw.filledCircle(closest[0], closest[1], .005);
		draw.show();
	}

	@Override
	public void mouseReleased(double x, double y) {
	}

	@Override
	public void mouseClicked(double x, double y) {
	}

	@Override
	public void keyTyped(char c) {
		keytypedbug = !keytypedbug;
		if (keytypedbug)
			return;

		if (c == '+')
			nbpoints = nbpoints + nbpoints / 2 + 1;
		else if (c == '-')
			nbpoints = nbpoints - nbpoints / 3;

		inittree();
		draw();
		draw.show();
	}

	@Override
	public void keyPressed(int keycode) {
	}

	@Override
	public void keyReleased(int keycode) {
	}

	static void trace(double[] point, double[] champion) {
		if (dotrace) {
			trace.addLast(point);
			champions.addLast(champion);
		}
	}
    
    static Draw draw;
	static boolean dotrace = false;
	static LinkedList<double[]> trace = new LinkedList<double[]>();
	static LinkedList<double[]> champions = new LinkedList<double[]>();
    
}

In [10]:
/*static Draw draw;
static boolean dotrace = false;
static LinkedList<double[]> trace = new LinkedList<double[]>();
static LinkedList<double[]> champions = new LinkedList<double[]>();*/

SwingUtilities.invokeLater(() -> {
    InteractiveClosest.draw = new Draw();
    InteractiveClosest.dotrace = true;
    InteractiveClosest.draw.setCanvasSize(700, 700);
    InteractiveClosest.draw.enableDoubleBuffering();
    InteractiveClosest obj = new InteractiveClosest(50);
    obj.draw();
    InteractiveClosest.draw.show();
    InteractiveClosest.draw.addListener(obj);
});
/*draw = new Draw();
dotrace = true;
draw.setCanvasSize(700, 700);
draw.enableDoubleBuffering();
InteractiveClosest obj = new InteractiveClosest(50);
obj.draw();
draw.show();
draw.addListener(obj);*/

## ColorPalette

In [11]:
// ColorPalette.java
public class ColorPalette {

	static Picture round64(Picture pic) {
		for (int i = 0; i < pic.width(); i++) {
			for (int j = 0; j < pic.height(); j++) {
				int c = pic.getRGB(i, j);
				int[] cc = new int[] { (c >> 16) & 255, (c >> 8) & 255, c & 255 };
				for (int k = 0; k < 3; k++)
					cc[k] = (int) (Math.round(cc[k] / 51.) * 51);
				pic.setRGB(i, j, (cc[0] << 16) | (cc[1] << 8) | cc[2]);
			}
		}
		return pic;
	}

	static Picture simplify(Picture pic, int maxpoints) {
		
		System.out.println("--Test of palette ... ");
		long startTime = System.currentTimeMillis();
		Random r = new Random(0);

		KDTree tree = null;
		int iter = 20000;
		for (int i = 0; i < iter; i++) {
			int row = r.nextInt(pic.height()), col = r.nextInt(pic.width());
			int c = pic.getRGB(col, row);
			// construction of the node labeled by a point which contains the 3 color coordinates
			double[] point = new double[3];
			point[0] = (c >> 16) & 255;
			point[1] = (c >> 8) & 255;
			point[2] = c & 255;
			tree = KDTree.insert(tree, point);
		}
		
		Vector<double[]> palette = KDTree.palette(tree, maxpoints);
		assert palette.size() == maxpoints: "the palette is not of the requested size";
		Collections.shuffle(palette); // to avoid degenerate trees
		tree = null;
		for (double[] c : palette)
			tree = KDTree.insert(tree, c);
		
		double[] tmp = new double[3];

		double score = 0.0;

		for (int i = 0; i < pic.width(); i++) {
			for (int j = 0; j < pic.height(); j++) {
				int c = pic.getRGB(i, j);
				tmp[0] = (c >> 16) & 255;
				tmp[1] = (c >> 8) & 255;
				tmp[2] = c & 255;
				double[] closest = KDTree.closest(tree, tmp);
				
				score += KDTree.sqDist(closest, tmp);

				pic.setRGB(i, j, ((int) closest[0] << 16) | ((int) closest[1] << 8) | (int) closest[2]);
	
			}
		}

		long endTime = System.currentTimeMillis();

		System.out.println("Simplifying the image takes " + (endTime - startTime) / 1000. + " seconds.");
		System.out.printf("The total score is %f.%n", Math.sqrt(score / pic.width() / pic.height()));
		
		return pic;
	}	
}

Initialisation avec 50 points.


In [12]:
//Witness Palette
Picture orig = new Picture("Figure 1.png");
orig.show();

//Palette web
Picture pic64 = ColorPalette.round64(new Picture(orig));
pic64.show();

//Palette optimized
Picture pic = ColorPalette.simplify(new Picture(orig), 256);
pic.show();	

--Test of palette ... 
Simplifying the image takes 3.584 seconds.
The total score is 13.033755.


# Tests

## TestAverage

In [13]:
// TestAverage.java
public class TestAverage {

	static void testSize(KDTree inputTree, int output) {
		assert (output == KDTree.size(inputTree)) : " the number of nodes should be " + output;

	}

	static void testSum(KDTree inputTree, double[] inputAcc, double[] outputAcc) {

		KDTree.sum(inputTree, inputAcc);

		for (int i = 0; i < inputTree.point.length; i++) {
			assert (Math.abs(outputAcc[i] - inputAcc[i]) < 1e-9) : "Error in sum calculation";
		}

	}

	static void testAverage(KDTree inputTree, double[] output) {

		double[] acc = new double[inputTree.point.length];

		KDTree.sum(inputTree, acc);
		int size1 = KDTree.size(inputTree);
		for (int i = 0; i < inputTree.point.length; i++) {
			assert (Math.abs(output[i] - acc[i] / size1) < 1e-9) : "Error in calculating the average at"
					+ " coefficient" + i;
		}
	}	
}

In [14]:
Picture pic = new Picture("photo.jpg");
// pic.show();

Random r = new Random();

KDTree tree = null;
int iter = 20000;
int size = KDTree.size(tree);

assert (size == 0) : "the number of nodes of tree is 0";

for (int i = 0; i < iter; i++) {
    int row = r.nextInt(pic.height()), col = r.nextInt(pic.width());
    int c = pic.getRGB(col, row);

    // construction of the node labeled by a point which contains the 3 color coordinates
    double[] point = new double[3];
    point[0] = (c >> 16) & 255;
    point[1] = (c >> 8) & 255;
    point[2] = c & 255;
    tree = KDTree.insert(tree, point);
}
System.out.println("--Test of the method size ... ");

TestAverage.testSize(tree, 20000);
double[] p0 = { -2.0, 0.0, 0.0 };
KDTree treeTest = KDTree.insert(null, p0);
TestAverage.testSize(treeTest, 1);
double[] p2 = { -2.0, 0.0, 0.0 };
double[] p1 = { -1.0, -1.0, 0.0 };

treeTest = KDTree.insert(treeTest, p1);

TestAverage.testSize(treeTest, 2);

System.out.println("--Test of the method sum ... ");

treeTest = KDTree.insert(treeTest, p2);

double[] inacc1 = new double[treeTest.point.length];

double[] outacc1 = new double[] { -5.0, -1.0, 0.0 };
TestAverage.testSum(treeTest, inacc1, outacc1);

System.out.println("--Test of the method average ... ");

double[] averagepoint = KDTree.average(treeTest);
TestAverage.testAverage(treeTest, averagepoint);

double p3[] = { 0.0, 0.5, 0.0 };

KDTree treeTest1 = new KDTree(p3, 1);
double p4[] = { -1.0, -1.0, -1.0 };
double p5[] = { -1.0, 6.0, 0.5 };

treeTest1 = KDTree.insert(treeTest1, p3);
treeTest1 = KDTree.insert(treeTest1, p4);
treeTest1 = KDTree.insert(treeTest1, p5);

double[] inacc2 = new double[treeTest1.point.length];

double[] outacc2 = new double[] { -2.0, 6.0, -0.5 };

TestAverage.testSum(treeTest1, inacc2, outacc2);

averagepoint = KDTree.average(treeTest1);
TestAverage.testAverage(treeTest1, averagepoint);

System.out.println("[OK]");

--Test of the method size ... 
--Test of the method sum ... 
--Test of the method average ... 
[OK]


## TestInsert

In [15]:
// TestInsert.java

public class TestInsert {
	
	/*public static int size(KDTree tree) {
		if (tree == null)
			return 0;
		return 1 + size(tree.left) + size(tree.right);
	}*/
	
	public static int size(KDTree tree) {
		if (tree == null)
			return 0;
		int result = 1;
		LinkedList<KDTree> queue = new LinkedList<KDTree>();
		queue.addLast(tree.left);
		queue.addLast(tree.right);
		while(!queue.isEmpty()){
			KDTree current = queue.pop();
			if (current == null)
				continue;
			queue.addLast(current.left);
			queue.addLast(current.right);
			result += 1;
		}
		return result;
	}

	static double p [][] = {
			{ 0.0, 0.0, 0.0 },
			{ -1.0, 0.0, 0.0 },
			{ 1.0, 0.0, 0.0 },
			{ 0.0, 2.0, 0.0 },
			{ -1.0, -1.0, -1.0 }
	};

	static KDTree do_insert(KDTree kd, int i) {
		System.out.printf("kd = insert(%s, p%d = [%f, %f, %f]);\n",
				(kd == null ? "null" : "kd"),
				i, p[i][0], p[i][1], p[i][2]);
		int size_before = size(kd);
		kd = KDTree.insert(kd, p[i]);
		int size_after = size(kd);
		assert (size_after == size_before + 1):
			String.format("size before last insertion = %d, after = %d", size_before, size_after);
		return kd;
	}
}


In [16]:
// test de KDtree.insert
System.out.println("--Test of the method insert");

KDTree kd = TestInsert.do_insert(null, 0);
assert (kd != null) : "...should return a new KDTree containing p at depth 0 but returned null";

kd = TestInsert.do_insert(kd, 1);
assert (kd.left != null) : "...devrait renvoyer un KDTree tel que kd.left!=null";
assert (kd.left.point.equals(TestInsert.p[1])) : "kd should contain point p1 in the left subtree";


kd = TestInsert.do_insert(kd, 2);
assert (kd.right != null) : "kd.right should be non-null";
assert (kd.right.point.equals(TestInsert.p[2])
        && kd.right.depth == 1) : "kd.right should be p2 at depth 1";

kd = TestInsert.do_insert(kd, 4);
assert (kd.left.left != null) : "kd.left.left should be != null";
assert (kd.left.left.point.equals(TestInsert.p[4])
        && kd.left.left.depth == 2) : "kd.left.left.point should be p4 at depth 2";

kd = TestInsert.do_insert(kd, 3);
kd = TestInsert.do_insert(kd, 4);
assert (kd.right != null) : "kd.right should be != null";
assert (kd.right.point.equals(TestInsert.p[2])
        && kd.right.depth == 1) : "kd.right should be p2 at depth 1";
assert (kd.right.right != null) : "kd.right.right should be != null";
assert (kd.right.right.point.equals(TestInsert.p[3])	&& kd.right.right.depth == 2) :
        "kd.right.right should be p3 at depth 2,"
        + " we have kd.right.right.depth=" + kd.right.right.depth;

System.out.println("[OK]");

--Test of the method insert
kd = insert(null, p0 = [0.000000, 0.000000, 0.000000]);
kd = insert(kd, p1 = [-1.000000, 0.000000, 0.000000]);
kd = insert(kd, p2 = [1.000000, 0.000000, 0.000000]);
kd = insert(kd, p4 = [-1.000000, -1.000000, -1.000000]);
kd = insert(kd, p3 = [0.000000, 2.000000, 0.000000]);
kd = insert(kd, p4 = [-1.000000, -1.000000, -1.000000]);
[OK]


## TestClosest

In [17]:
// TestClosest.java
public abstract class TestClosest {

	String name;

	abstract double[] closest(KDTree tree, double[] point);

	static String pointToString(KDTree tree, double[] point) {
		String res = String.format(Locale.ROOT, "[%f, %f, %f]", point[0], point[1], point[2]);
		if (tree != null) {
			if (Arrays.equals(point, tree.point))
				res += " (= root of the tree)";
			else if (tree.left != null && Arrays.equals(point, tree.left.point))
				res += " (= tree.left.point)";
			else if (tree.right != null && Arrays.equals(point, tree.right.point))
				res += " (= tree.right.point)";
		}
		return res;
	}

	public void testClosest(int size, int runs) {

		if (!TestClosest.class.desiredAssertionStatus()) {
			System.err.println("You must pass the -ea option to the Java Virtual Machine.");
			System.err.println("(Run As -> Run configurations -> Arguments -> VM Arguments)");
			System.exit(1);
		}

		// we fill a 3d tree with the colors of pixels of an image,
		// and we get a subset of the same points to use in the tests
		Picture pic = new Picture("photo.jpg");
		int height = pic.height();
		int width = pic.width();
		int stride = height * width / size;
		assert size >= 0 && stride > 0 : "Parameter size incorrect";
		assert runs <= size : "runs should be <= size";
		int testStride = (size - 3) / (runs - 3);

		KDTree tree = null;
		Vector<double[]> testPoints = new Vector<double[]>();
		for (int i = 0; i < size; i++) {
			int c = pic.getRGB((i * stride) % width, i * stride / width);
			double[] point = new double[3];
			point[0] = (c >> 16) & 255;
			point[1] = (c >> 8) & 255;
			point[2] = c & 255;
			tree = KDTree.insert(tree, point);
			if (i < 3 || (i - 3) % testStride == 0 && testPoints.size() < runs)
				testPoints.add(point);
		}
		
		assert TestInsert.size(tree) == size: "impossible to build the test tree (insert() method incorrect?)";

		System.out.printf("--Test of the method %s (%d points, 16*%d tests)...\n", name, size, runs);

		long startTime = System.currentTimeMillis();

		for (double[] a : testPoints) {
			// we use here the fact that by construction, the points of
			// our test tree have integer coordinates
			for (int i = 0; i < 8; i++) {
				double[] b = a.clone();
				for (int j = 0; j < 3; j++)
					if ((i & (1 << j)) != 0)
						b[j] += .1;
				double[] c = closest(tree, b);
				assert (Arrays.equals(a, c)) : String.format("the nearest point of %s should be %s and not %s",
						pointToString(tree, b), pointToString(tree, a), pointToString(tree, c));
			}
		}
		
		// second test with slightly larger perturbations
		Random rnd = new Random(0);
		for (double[] a : testPoints) {
			for (int i = 0; i < 8; i++) {
				double[] b = a.clone();
				for (int j = 0; j < 3; j++)
					b[j] += rnd.nextGaussian()/2;
				double[] c = closest(tree, b);
				assert (KDTree.sqDist(c, b) <= KDTree.sqDist(a, b)) : String.format("The nearest point of %s is not %s (%s is the closest)",
						pointToString(tree, b), pointToString(tree, c), pointToString(tree, a));
			}
		}

		long endTime = System.currentTimeMillis();
		System.out.printf("Total time : %f sec.\n", (endTime - startTime) / 1000.);
		System.out.println("[OK]");

	}
}

## TestClosestNaive

In [18]:
// TestClosestNaive.java
public class TestClosestNaive extends TestClosest {

	TestClosestNaive() {
		name = "closestNaive()";
	}

	double[] closest(KDTree tree, double[] a) {
		return KDTree.closestNaive(tree, a);
	}
}


In [19]:
TestClosest test = new TestClosestNaive();
test.testClosest(10, 10);
test.testClosest(100, 100);
System.out.println("the following test may take a few seconds");	
test.testClosest(20000, 1000);

--Test of the method closestNaive() (10 points, 16*10 tests)...
Total time : 0.004000 sec.
[OK]
--Test of the method closestNaive() (100 points, 16*100 tests)...
Total time : 0.016000 sec.
[OK]
the following test may take a few seconds
--Test of the method closestNaive() (20000 points, 16*1000 tests)...
Total time : 23.584000 sec.
[OK]


## TestClosestOptimized

In [20]:
// TestClosestOptimized.java
public class TestClosestOptimized extends TestClosest {

	TestClosestOptimized() {
		name = "closest() [optimized version]";
	}

	double[] closest(KDTree tree, double[] a) {
		return KDTree.closest(tree, a);
	}
}


In [21]:
TestClosestOptimized test = new TestClosestOptimized();
test.testClosest(10, 10);
test.testClosest(100, 100);
test.testClosest(20000, 1000);
System.out.println("the following test may take a few seconds");	
test.testClosest(100000, 100000);

--Test of the method closest() [optimized version] (10 points, 16*10 tests)...
Total time : 0.000000 sec.
[OK]
--Test of the method closest() [optimized version] (100 points, 16*100 tests)...
Total time : 0.002000 sec.
[OK]
--Test of the method closest() [optimized version] (20000 points, 16*1000 tests)...
Total time : 0.017000 sec.
[OK]
the following test may take a few seconds
--Test of the method closest() [optimized version] (100000 points, 16*100000 tests)...
Total time : 0.749000 sec.
[OK]


## TestCompare

In [22]:
// TestCompare.java
// test de KDtree.compare
System.out.println("--Test of the method compare ...");

double[][] testPoints = {
        {0., 0., 0.},
        {-.42, 7.5765, -3.7}
};

for (double[] a : testPoints) {
    for (int d = 0; d < 3*a.length; d++) {
        assert a.length >= 2;
        KDTree kd = new KDTree(a, d);
        double[] b = a.clone();
        assert kd.compare(b):
            String.format("t.compare(%s) for a subtree t from root %s to depth %d should return true",
                    Arrays.toString(b), Arrays.toString(a), d);
        b[d % a.length] += 1.;
        assert kd.compare(b):
            String.format("t.compare(%s) for a subtree t from root %s to depth %d should return true",
                    Arrays.toString(b), Arrays.toString(a), d);
        b[(d + 1) % a.length] += 1.;
        assert kd.compare(b):
            String.format("t.compare(%s) for a subtree t from root %s to depth %d should return true",
                    Arrays.toString(b), Arrays.toString(a), d);
        b[d % a.length] -= 2.;
        assert !kd.compare(b):
            String.format("t.compare(%s) for a subtree t from root %s to depth %d should return false",
                    Arrays.toString(b), Arrays.toString(a), d);
    }
}

System.out.println("[OK]");

--Test of the method compare ...
[OK]


## TestSqDist (code to complete)

In [23]:
// TestSqDist.java
public class TestSqDist {
    static void testSqDist(double[] a, double[] b, double expected) {     
        double d = KDTree.sqDist(a, b);
        assert (d == expected) : String.format("sqDist should return %f but returns %f", expected, d);
    }
}


In [24]:

System.out.println("--Test of the method sqDist ...");
TestSqDist.testSqDist(new double[] {-1.0, 1.0}, new double[] {1.0,-1.0}, 8.0);
TestSqDist.testSqDist(new double[] {}, new double[] {}, 0);                      // 0D
TestSqDist.testSqDist(new double[] {1}, new double[] {4}, 9);                    // 1D
TestSqDist.testSqDist(new double[] {0, 0}, new double[] {3, 4}, 25);             // 2D
TestSqDist.testSqDist(new double[] {1, 2, 3}, new double[] {4, 5, 6}, 27);       // 3D
TestSqDist.testSqDist(new double[] {1, 0, -1}, new double[] {-1, 0, 1}, 8);      // 3D with negatives
TestSqDist.testSqDist(new double[] {0, 0, 0, 0}, new double[] {1, 1, 1, 1}, 4);  // 4D
TestSqDist.testSqDist(new double[] {5, 5}, new double[] {5, 5}, 0);              // same point
TestSqDist.testSqDist(new double[] {1, 2, 3, 4, 5}, new double[] { 5, 4, 3, 2, 1}, 40); // 5D
System.out.println("[OK]");

--Test of the method sqDist ...
[OK]
